In [ ]:
!pip install --upgrade xlrd
!pip install bizdays
!pip install pyathena
!pip install --upgrade google-cloud-bigquery
!pip install db_dtypes
!pip install python-dateutil
!pip install pandas-gbq
!pip install  xlsxwriter
!pip install selenium
!pip install pandas==1.4.0 

In [ ]:
import gspread  ## https://docs.gspread.org/en/v5.4.0/
from google.auth import default
from google.colab import data_table
from google.oauth2 import service_account
from google.cloud import bigquery
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import files
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import pandas as pd
from datetime import date, timedelta
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import data_table
#import bizdays
data_table.enable_dataframe_formatter()
import numpy as np
#from bizdays import set_option
#set_option('mode', 'pandas')
from google.oauth2 import service_account
from google.cloud import bigquery
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#import db_dtypes
import os
from pyarrow import *
import json
import datetime
import dateutil
from dateutil import parser
from google.colab import files
import xlsxwriter
import math
#from bcb import sgs


In [ ]:
import os, time
#print(time.strftime('%Y-%m-%d %H:%M:%S')) # before timezone change
os.environ['TZ'] = 'Brazil/East' # set new timezone
time.tzset()
pd.set_option('display.float_format', lambda x: '%.8f' % x)
pd.set_option("display.precision", 8)
from google.colab import auth
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
pd.set_option('use_inf_as_na',True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
local = '/content/drive/MyDrive/Configs/XXXXXXXXXXXXXXXXXXX'
local_salvo_chrome_driver = '/content/drive/MyDrive/Configs/chromedriver.exe'

Mounted at /content/drive


In [ ]:
#Configurando os componentes da query de posição
local = '/content/drive/MyDrive/Configs/XXXXXXXXXXXXXXXXXXXXXX'
dataset_id = 'XXXXXXXXXXXX'
project_id = 'XXXXXXXXXXXXXXXXXXXX'
credentials = service_account.Credentials.from_service_account_file(local)


client = bigquery.Client(credentials= credentials,project=project_id)
dataset_ref = client.dataset(dataset_id)

# Funções


In [ ]:
def concat(*args):
    return pd.concat([x for x in args if not x.empty])


def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()


def aliquota_ir(dcs_vencimento):
  if dcs_vencimento <= 180:
    aliquota = 0.225
  elif dcs_vencimento <= 360:
    aliquota = 0.20
  elif dcs_vencimento <= 720:
    aliquota = 0.175
  elif dcs_vencimento > 720:
    aliquota = 0.15

  return aliquota

def handle_nan(value):
    if math.isnan(value):
        return 0
    else:
        return value



def breakeven(FV,PV,taxa):
  breakeven = handle_nan((np.log(FV) - np.log(PV))/np.log(1+taxa))
  return int(breakeven*360)




In [ ]:

def datechange(datestr):
    diaobj = datestr.split('T')
    completo =diaobj[0]
    data_ajuste = completo.split('-')
    d = data_ajuste[2]
    m = data_ajuste[1]
    y = data_ajuste[0]
    return d+"/"+m+"/"+y


In [ ]:
def cdi_to_pre(taxa,di_futuro):
  """
        converte taxa %CDI em uma Taxa PRE a.a
  """
  equivalencia_dia = round((1+di_futuro)**(1/252),9)-1
  equivalencia_ano = (((equivalencia_dia*taxa)+1)**252)-1
  return round(equivalencia_ano,8)


def ipca_to_pre(taxa_ipca,implicita):
  pre = (1+taxa_ipca)*(1+implicita)-1
  return round(pre,8)

def cdi_mais_to_pre(taxa,di_futuro):
  pre = (1+taxa)*(1+di_futuro)-1
  return round(pre,8)



def pre_to_ipca(rentabilidade,implicita):
  cupom = (1+rentabilidade)/(1+implicita)-1
  return round(cupom,4)

def pre_to_cdi(rentabilidade,di_futuro):
  rentabilidade_ajuste = round(((1+rentabilidade)**(1/252))-1,8)
  di_futuro_ajuste = round((1+di_futuro)**(1/252)-1,8)
  cdi_equivalente = round(rentabilidade_ajuste/di_futuro_ajuste,4)
  return cdi_equivalente


def pre_to_cdi_mais(rentabilidade,di_futuro):
  cupom = round((1+rentabilidade)/(1+di_futuro)-1,4)
  return cupom

In [ ]:
"""Scrapping"""
def click_ativos():
    wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='page-content']")))
    ativos = driver.find_elements_by_xpath("//a[@title='Expand']")[0]
    #for ativo in ativos:
    ativos.click()
        
    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.HOME) 
print("*"*100)
print("Passar Para Próxima Instância")
print("*"*100)

****************************************************************************************************
Passar Para Próxima Instância
****************************************************************************************************


# Datas Funções

In [ ]:
def feriados_anbima():
    """_summary_ busca lista de feriados padrão Brazil/ANBIMA


    Returns:
        dataframe: dataframe com lista de feriados até 2071
    """
    
    df = pd.read_excel(r'https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls')
    fer = list(df['Data'][0:len(df)-10])
    fer = [fer[i].date() for i in range(len(fer))]
    return fer



def dias_uteis(df, data_inicio,data_fim = None, incluir_data_inicio = False, hol=None):
    """_summary_: Calcular quantidade de dias uteis entre uma data inicio e datafim
                    Caso data_fim nao especificada, será considerado como dia de hoje.
                    Caso incluir_data_inicio não especificado, não será considerado o dia inicio no calculo.

    Args:
        data_inicio (str): dia inicial para o calculo.
        data_fim (str, optional): data_fim para o calculo, caso não especificado será considerado o dia de hoje.
        incluir_data_inicio (bool, optional): False: inclui a data inicio no calculo. True: para caso queira considerar. Defaults to False.
        hol (list, optional): lista de feriados. Defaults to None.

    Returns:
        numpy.int64: quantidade de dias uteis entre duas datas.
    """
    
  
    data_inicio_ajustada = ajustar_data(data_inicio)

    
    if data_fim == None:
        data_fim_ajustada = ajustar_data(date.today())
    else:  
        data_fim_ajustada =  ajustar_data(data_fim)
        
    if hol==None: hol = df
    
    if incluir_data_inicio is False:

      return np.busday_count(data_inicio_ajustada.date(), data_fim_ajustada.date(), holidays=hol)
            
    else:

      return np.busday_count(data_inicio_ajustada.date(), data_fim_ajustada.date(), holidays=hol)+1

def ajustar_data(data_ajustar,return_str = False):
        """Ajusta data para formato YYYY-MM-DD

        Args:
            data_ajustar (date): data a ser ajustada

        Returns:
            date: data no formato YYYY-MM-DD
        """

        data_ajustar = parser.parse(str(data_ajustar),dayfirst=True)  # datetime.datetime(1999, 8, 28, 0, 0)
        data_inicio_y = data_ajustar.year
        data_inicio_m = data_ajustar.month
        data_inicio_d = data_ajustar.day
        if data_inicio_m<10: data_inicio_m='0'+str(data_inicio_m)
        if data_inicio_d<10: data_inicio_d = '0'+str(data_inicio_d)
        data_ajustada = f'{data_inicio_y}-{data_inicio_m}-{data_inicio_d}'

        
        if return_str is False:
            data_ajustada = parser.parse(str(data_ajustada),dayfirst=True)
            #print(data_ajustada.date())
            return data_ajustada 
        else:
            return data_ajustada
fer = feriados_anbima()





def verifica_data_aniversario_fim(data_aniversario_fim):
  # Obtém a data atual
  hoje = datetime.today()

  # Verifica se a data de aniversário final é maior que a data atual
  if data_aniversario_fim > hoje:
    # Se a data de aniversário final for maior, subtrai um ano da data
    data_aniversario_fim = data_aniversario_fim.replace(year=data_aniversario_fim.year-1)

  # Retorna a data de aniversário final atualizada
  return data_aniversario_fim


# B3 Funcs

In [ ]:
class b3_:
    def __init__(self, data_val):
        """
        Gera o objeto da b3 a partir de data_val em formato de datetime.date
        """
        self.data_val = data_val
        mes = self.data_val.month
        dia = self.data_val.day
        if self.data_val.month<10: mes='0'+str(self.data_val.month)
        if self.data_val.day<10: dia = '0'+str(self.data_val.day)
        self.dt_barra = f'{dia}/{mes}/{data_val.year}'
        self.dt_corrida = f'{data_val.year}{mes}{dia}'
        self.headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

    def _baixar_pre(self):
      
        

        link = f'https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data={self.dt_barra}&Data1={self.dt_corrida}&slcTaxa=PRE'
        page = requests.get(link, verify=False)
        soup = BeautifulSoup(page.content, 'html.parser')
        texto = soup.find_all('td')
        dias, taxas252, taxas360 = [], [], []
        tables = ["['tabelaConteudo1']", "['tabelaConteudo2']"]
        for i in range(0,len(texto),3):
            try:
                  if i<=len(texto)-2:
                      dias.append(int(texto[i].text.replace('\r\n','').replace(',','.').replace(' ','')))
                      taxas252.append(float(texto[i+1].text.replace('\r\n','').replace(',','.').replace(' ',''))/100)
                      taxas360.append(float(texto[i+2].text.replace('\r\n','').replace(',','.').replace(' ',''))/100)
            except:
                pass
        return pd.DataFrame({'taxas252':taxas252,'taxas360':taxas360}, index=dias)   
    
    
    
    def _baixa_cupom_ipca(self):
        """
        Dada a val_date baixa a curva de cupom limpo
        """
        #link = 'https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data=16/09/2022&Data1=20220916&slcTaxa=DIC#'
        link = f'https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data={self.dt_barra}&Data1={self.dt_corrida}&slcTaxa=DIC'
        print(link)
        page = requests.get(link, verify=False)
        soup = BeautifulSoup(page.content, 'html.parser')
        texto = soup.find_all('td')
        dias, taxas = [], []
        tabelas = ["['tabelaConteudo1']", "['tabelaConteudo2']"]
        
        for i in range(len(texto)):
            try:
                  tratado = texto[i].text.replace('\r\n','').replace(',','.').replace(' ','')
                  if i==0 or i%2==0:
                      dias.append(int(tratado))
                  else:
                      taxas.append(float(tratado)/100)
            except:
                pass

        return pd.DataFrame(data=taxas, index=dias, columns={'taxas252'})
        

In [ ]:
def get_SGS(codigo_serie,data_inicial,data_final=None):


  DataInicio = data_inicial
  DataFim = date.today() - timedelta(4)
  DataFim = DataFim.strftime("%d/%m/%Y")
  print(DataFim)

  codigo = codigo_serie
  link = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=json&dataInicial={DataInicio}&dataFinal={DataFim}'
  page = requests.get(link, verify=False)
  dados = pd.read_json(link)
  return dados

In [ ]:
def _df_CDI():
  df_cdi_bcb = get_SGS(12,'01/01/2000')
  df_cdi_bcb.rename(columns = {'data':'Date', 'valor':'CDI'}, inplace = True)
  df_cdi_bcb['CDI'] = df_cdi_bcb.CDI/100
  df_cdi_bcb.Date = df_cdi_bcb.Date.apply(lambda _: datetime.datetime.strptime(_,"%d/%m/%Y").date())
  return df_cdi_bcb
df_cdi_bcb = _df_CDI()
#df_cdi_bcb

08/04/2023


# Yield Funcs

In [ ]:
"""
Criar data para consulta
"""

hoje = date.today() - timedelta(1)
while hoje.weekday() > 4 or hoje in fer:
  hoje = hoje - timedelta(1)

ano = hoje.year
mes = hoje.month
dia= hoje.day


"""
Criar IPCA Interpolado
"""

dados_ipca = b3_(date(ano,mes,dia))._baixa_cupom_ipca()
while len(dados_ipca) < 1:
    hoje = hoje - timedelta(1)
    ano = hoje.year
    mes = hoje.month
    dia= hoje.day
    dados_ipca = b3_(date(ano,mes,dia))._baixa_cupom_ipca()
datas_ipca = []

datas_ipca = list(range(1,36000))
#last_element = int(dados_pre.iloc[-1])
#for i in range(0,3600+1):
#  datas.append(i)
datas_ipca = pd.DataFrame(datas_ipca)


df_ipca = pd.concat([datas_ipca, dados_ipca], axis=1)
df_ipca.set_index(df_ipca.columns[0])
ser1 = df_ipca['taxas252']
ser1 = ser1.interpolate(method='polynomial', order=2)
ipca_dados = ser1.to_frame()
ipca_dados.dropna(inplace=True)
ipca_dados = ipca_dados.head(36000)
ipca_interpolado = ipca_dados.rename(columns={"taxas252":"Cupom-IPCA"})


"""
Criar PRE Interpolado
"""

dados_pre = b3_(date(ano,mes,dia))._baixar_pre()
while len(dados_pre) < 1:
    hoje = hoje - timedelta(1)
    ano = hoje.year
    mes = hoje.month
    dia= hoje.day
    dados_pre = b3_(date(ano,mes,dia))._baixar_pre()

dados_pre.drop('taxas360', axis=1, inplace=True)

datas = list(range(1,36000))
#last_element = int(dados_pre.iloc[-1])
#for i in range(0,3600+1):
#  datas.append(i)
datas = pd.DataFrame(datas)  

df2 = pd.concat([datas, dados_pre], axis=1)
df2.set_index(df2.columns[0])
ser1 = df2['taxas252']
ser1 = ser1.interpolate(method='polynomial', order=2)
dif = ser1.to_frame()
dif.dropna(inplace=True)
dif = dif.head(36000)
pre_interpolado = dif.rename(columns={"taxas252":"DI-1"})


"""
Criar Inflação Implicita Interpolada
"""

dfs_implicita = [pre_interpolado, ipca_interpolado]

result_implicita = pd.concat(dfs_implicita, join='inner', axis=1).fillna(np.nan)
result_implicita['Implicita'] = ((1+result_implicita['DI-1'])/(1+result_implicita['Cupom-IPCA']))-1
implicita_interpolada = result_implicita['Implicita']





https://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data=11/04/2023&Data1=20230411&slcTaxa=DIC


In [ ]:
sheets = 'Master Proprietaria'
print('abrindo sheets vna')
sh = gc.open(sheets)
df_ipca_vna = get_as_dataframe(sh.worksheet('IPCA'), parse_dates=True, evaluate_formulas=True)
df_ipca_vna=df_ipca_vna.dropna(how='all')
df_ipca_vna=df_ipca_vna.dropna(axis=1, how='all')
df_ipca_vna['DATA'] = pd.to_datetime(df_ipca_vna['DATA'], dayfirst=True)
df_ipca_vna['DATA'] = pd.to_datetime(df_ipca_vna['DATA']).apply(lambda x: x.date())
df_ipca_vna

In [ ]:
def acumulo_cdi(data_inicio,taxa,df_cdi):
  #data_inicio_y = data_inicio.year
  #print(f'essa é a data inicio{data_inicio_y}')
  #data_inicio_m = data_inicio.month
  #data_inicio_d = data_inicio.day
  #if data_inicio_m<10: data_inicio_m='0'+str(data_inicio_m)
  #if data_inicio_d<10: data_inicio_d = '0'+str(data_inicio_d)
  #data_inicio = f'{data_inicio_y}-{data_inicio_m}-{data_inicio_d}'


  data_hoje = date.today()
  data_hoje_y = data_hoje.year
  data_hoje_m = data_hoje.month
  data_hoje_d = data_hoje.day
  if data_hoje_m<10: data_hoje_m='0'+str(data_hoje_m)
  if data_hoje_d<10: data_hoje_d = '0'+str(data_hoje_d)
  data_hoje = f'{data_hoje_y}-{data_hoje_m}-{data_hoje_d}'

  #data_inicio = data_inicio.apply(lambda _: datetime.datetime.strptime(_,"%d/%m/%Y").date())
  data_inicio = data_inicio.date()
  taxa = round(float(taxa),2)
  print(data_inicio)
  print(type(data_inicio))
  print(data_hoje)
  print(taxa)

  df_temp = df_cdi.copy()
  df_temp = df_temp.query('Date >= @data_inicio')
  df_temp.reset_index(inplace=True)
  df_temp['Fator'] = taxa

  df_temp['Ajuste'] = df_temp.CDI*df_temp.Fator
  df_temp['Dados'] = 1+df_temp.Ajuste
  #df2.reset_index(inplace=True)
  df_temp['Cumulative']= round(df_temp.Dados.cumprod(),8)
  resposta = df_temp['Cumulative'].values[-1]
  print(resposta)


  #link = f'https://calculadorarendafixa.com.br/calculadora/di/calculo?dataInicio={data_inicio}&1&dataFim={data_hoje}&percentual={taxa}&valor=1000.00'
  #page = requests.get(link)
  #soup = BeautifulSoup(page.content, 'html.parser')
  #site_json=json.loads(soup.text) 
  #site_json


  return float(resposta)

# Grossup

In [ ]:
def grossup_isento(df_pre,df_implicita,df_feriados, produto,dias_corridos,taxa,indexador):
    """Realiza o Grossup dos Ativos, considerando dias_corridos e o indexador correspondente.

    Args:
        produto (str): Instrumento Financeiro, CDB e LC Nao Isento, LCA e LCI Isentos
        dias_corridos (int): dias corridos - ex: 1800
        taxa (float): taxa do ativo ex: 15% = 15
        indexador (str): Indexador do Ativo: CDI+,CDI,IPCA,PRE

    Returns:
        : _description_
    """
    taxa = taxa/100
    lista_isentos = ['LCA','LCI']
    prazo = dias_corridos
    lista_index_especiais = ['IPCA','CDI+']
    if indexador not in lista_index_especiais:
        if produto in lista_isentos:
            if dias_corridos < 180:
                taxa_grossup = round(taxa/0.775,4)
            elif dias_corridos >= 180 and dias_corridos < 360:
                taxa_grossup = round(taxa/0.80,4)
            elif dias_corridos >= 360 and dias_corridos < 720:
                taxa_grossup = round(taxa/0.825,4)
            elif dias_corridos >= 720:
                taxa_grossup = round(taxa/0.85,4)
                
            return taxa_grossup
        else: return round(taxa,4)
        
    elif indexador == 'IPCA':
        if produto in lista_isentos:
            hoje = date.today()
            vencimento = hoje + timedelta(days=dias_corridos)
            inf_implicita = implicita_interpolada[dias_corridos]
            dus = dias_uteis(df_feriados,hoje,vencimento)
            rentabilidade_pre = round((1+taxa)**(dus/252),8)
            rentabilidade_implicita = round((1+inf_implicita)**(dus/252),4)
            rentabilidade_total = rentabilidade_pre*rentabilidade_implicita-1
            if dias_corridos < 180:
                taxa_grossup = round(rentabilidade_total/0.775,4)
            elif dias_corridos >= 180 and dias_corridos < 360:
                taxa_grossup = round(rentabilidade_total/0.80,4)
            elif dias_corridos >= 360 and dias_corridos < 720:
                taxa_grossup = round(rentabilidade_total/0.825,4)
            elif dias_corridos >= 720:
                taxa_grossup = round(rentabilidade_total/0.85,4)

            taxa_grossup = round(((1+taxa_grossup)**(252/dus)/(1+inf_implicita))-1,4)
            return taxa_grossup
        else:
            return round(taxa,4)

    elif indexador == 'CDI+':
        if produto in lista_isentos:
            hoje = date.today()
            vencimento = hoje + timedelta(days=dias_corridos)
            di_futuro_ = pre_interpolado['DI-1'][vencimento]
            di_futuro_ = di_futuro_[0]
            dus = dias_uteis(df_feriados,hoje,vencimento)
            rentabilidade_pre = round((1+taxa)**(dus/252),8)
            rentabilidade_implicita = round((1+di_futuro_)**(dus/252),4)
            rentabilidade_total = rentabilidade_pre*rentabilidade_implicita-1
            if dias_corridos < 180:
                taxa_grossup = round(rentabilidade_total/0.775,4)
            elif dias_corridos >= 180 and dias_corridos < 360:
                taxa_grossup = round(rentabilidade_total/0.80,4)
            elif dias_corridos >= 360 and dias_corridos < 720:
                taxa_grossup = round(rentabilidade_total/0.825,4)
            elif dias_corridos >= 720:
                taxa_grossup = round(rentabilidade_total/0.85,4)

            taxa_grossup = round(((1+taxa_grossup)**(252/dus)/(1+di_futuro_))-1,4)
            return taxa_grossup
        else:
            return round(taxa,4)



In [ ]:
def grossup_nao_isento(df_pre,df_implicita,df_feriados, produto,dias_corridos,taxa,indexador):
    """Realiza o Grossup dos Ativos, considerando dias_corridos e o indexador correspondente.

    Args:
        produto (str): Instrumento Financeiro, CDB e LC Nao Isento, LCA e LCI Isentos
        dias_corridos (int): dias corridos - ex: 1800
        taxa (float): taxa do ativo ex: 15% = 15
        indexador (str): Indexador do Ativo: CDI+,CDI,IPCA,PRE

    Returns:
        : _description_
    """
    taxa = taxa/100
    lista_isentos = ['LCA','LCI']
    prazo = dias_corridos
    lista_index_especiais = ['IPCA','CDI+']
    if indexador not in lista_index_especiais:
        if produto in lista_isentos:
            if dias_corridos < 180:
                taxa_grossup = round(taxa*0.775,4)
            elif dias_corridos >= 180 and dias_corridos < 360:
                taxa_grossup = round(taxa*0.80,4)
            elif dias_corridos >= 360 and dias_corridos < 720:
                taxa_grossup = round(taxa*0.825,4)
            elif dias_corridos >= 720:
                taxa_grossup = round(taxa*0.85,4)
                
            return taxa_grossup
        else: return round(taxa,4)
        
    elif indexador == 'IPCA':
        if produto in lista_isentos:
            hoje = date.today()
            vencimento = hoje + timedelta(days=dias_corridos)
            inf_implicita = implicita_interpolada[dias_corridos]
            dus = dias_uteis(df_feriados, hoje,vencimento)
            rentabilidade_pre = round((1+taxa)**(dus/252),8)
            rentabilidade_implicita = round((1+inf_implicita)**(dus/252),4)
            rentabilidade_total = rentabilidade_pre*rentabilidade_implicita-1
            if dias_corridos < 180:
                taxa_grossup = round(rentabilidade_total*0.775,4)
            elif dias_corridos >= 180 and dias_corridos < 360:
                taxa_grossup = round(rentabilidade_total*0.80,4)
            elif dias_corridos >= 360 and dias_corridos < 720:
                taxa_grossup = round(rentabilidade_total*0.825,4)
            elif dias_corridos >= 720:
                taxa_grossup = round(rentabilidade_total*0.85,4)

            taxa_grossup = round(((1+taxa_grossup)**(252/dus)/(1+inf_implicita))-1,4)
            return taxa_grossup
        else:
            return round(taxa,4)

    elif indexador == 'CDI+':
        if produto in lista_isentos:
            hoje = date.today()
            vencimento = hoje + timedelta(days=dias_corridos)
            di_futuro_ = pre_interpolado['DI-1'][dias_corridos]
            di_futuro_ = di_futuro_[0]
            dus = dias_uteis(df_feriados,hoje,vencimento)
            rentabilidade_pre = round((1+taxa)**(dus/252),8)
            rentabilidade_implicita = round((1+di_futuro_)**(dus/252),4)
            rentabilidade_total = rentabilidade_pre*rentabilidade_implicita-1
            if dias_corridos < 180:
                taxa_grossup = round(rentabilidade_total*0.775,4)
            elif dias_corridos >= 180 and dias_corridos < 360:
                taxa_grossup = round(rentabilidade_total*0.80,4)
            elif dias_corridos >= 360 and dias_corridos < 720:
                taxa_grossup = round(rentabilidade_total*0.825,4)
            elif dias_corridos >= 720:
                taxa_grossup = round(rentabilidade_total*0.85,4)

            taxa_grossup = round(((1+taxa_grossup)**(252/dus)/(1+di_futuro_))-1,4)
            return taxa_grossup
        else:
            return round(taxa,4)

# Funções de Cálculo

In [ ]:
def calculo_cdi_venda(pre_df, df_feriados,df_cdi):
  linhas = len(df_cdi)
  df_cdi['fd_datemissao'] = pd.to_datetime(df_cdi['fd_datemissao'], dayfirst=True)
  df_cdi['Vencimento'] = pd.to_datetime(df_cdi['Vencimento'], dayfirst=True)
  df_cdi['pu_curva'] = np.nan
  #df_cdi['taxa_compra'] = np.nan
  df_cdi['pu_compra'] = np.nan
  df_cdi['valor_total_compra'] = np.nan
  #df_cdi['taxa_venda'] = np.nan
  df_cdi['pu_venda'] = np.nan
  df_cdi['valor_total_venda'] = np.nan
  df_cdi['receita_mesa'] = np.nan
  df_cdi['receita_assessor'] = np.nan
  lista_ativo = ['LCA','LCI']
  x=0
  for x in range(0,linhas):
    taxa_emissao = df_cdi['taxa_emissao'][x]/100
    quantidade = int(df_cdi['qt_total'][x])
    pu_emissao = float(df_cdi['pu_emissao'][x])
    data_emissao = df_cdi.fd_datemissao[x]
    data_vencimento = df_cdi.Vencimento[x]
    time.sleep(0.5)
    print(f'data emissao:{data_emissao}')
    fator = acumulo_cdi(data_emissao,taxa_emissao,df_cdi_bcb)
    print(f'data emissao:{fator}')
    pu_curva = fator * pu_emissao
    print(f'data emissao:{pu_curva}')
    df_cdi.at[x,'pu_curva']= pu_curva
    valor_total = pu_curva * quantidade
    du_total = dias_uteis(df_feriados, data_emissao,data_vencimento)
    du_restante = dias_uteis(df_feriados, date.today(),data_vencimento)
    #vencimento_date = date(data_vencimento.year,data_vencimento.month,data_vencimento.day)
    print('estou aqui antes do vencimento')
    dc_vencimento = (date(data_vencimento.year,data_vencimento.month,data_vencimento.day) -date.today()).days
    if dc_vencimento < 3 and dc_vencimento >= 0:
      dc_vencimento = 10
      print('estou vencendo hoje!')
    pre_vencimento = pre_interpolado['DI-1'][dc_vencimento]


    taxa_compra = df_cdi['taxa_compra_sem_grossup'][x]/100
    print(taxa_compra)
    calculo_compra_1 = round(pu_curva*((((((1+pre_vencimento)**(1/252))-1)*taxa_emissao)+1)**du_restante),8)
    calculo_compra_2 = round(((((((1+pre_vencimento)**(1/252))-1)*taxa_compra)+1)**du_restante),8)
    calculo_compra = round(calculo_compra_1/calculo_compra_2,8)
    df_cdi.at[x,'pu_compra']= str(calculo_compra)
    df_cdi.at[x,'valor_total_compra']= round(calculo_compra * quantidade,2)
    receita_compra = pu_curva - calculo_compra
    taxa_venda = df_cdi['taxa_venda_sem_grossup'][x]/100
    calculo_venda_1 = pu_curva*((((((1+pre_vencimento)**(1/252))-1)*taxa_emissao)+1)**du_restante)
    calculo_venda_2 = ((((((1+pre_vencimento)**(1/252))-1)*taxa_venda)+1)**du_restante)
    calculo_venda = round(calculo_venda_1/calculo_venda_2,8)
    df_cdi.at[x,'pu_venda']= str(calculo_venda)
    df_cdi.at[x,'valor_total_venda']= round(calculo_venda * quantidade,2)
    receita_assessor = round((calculo_venda*quantidade)*((1+0.005)**(du_restante/252)-1),4)
    df_cdi.at[x,'receita_mesa']= str((calculo_venda - calculo_compra)*quantidade)
    df_cdi.at[x,'receita_assessor']= str(receita_assessor)
    print(f'estou finalizando o x = {x}')

  #print('estou ajustando os replaces!!')
  df_cdi['taxa_emissao'] = df_cdi['taxa_emissao'].astype(str).str.replace('.', ",")
  df_cdi['Vencimento']= pd.to_datetime(df_cdi['Vencimento']).apply(lambda x: x.date())
  df_cdi['fd_datemissao']= pd.to_datetime(df_cdi['fd_datemissao']).apply(lambda x: x.date())
  df_cdi['pu_emissao'] = df_cdi['pu_emissao'].astype(str).str.replace('.', ",")
  df_cdi['pu_curva'] = df_cdi['pu_curva'].astype(str).str.replace('.', ",")
  df_cdi['taxa_compra'] = df_cdi['taxa_compra'].astype(str).str.replace('.', ",")
  df_cdi['pu_compra'] = df_cdi['pu_compra'].astype(str).str.replace('.', ",")
  df_cdi['valor_total_compra'] = df_cdi['valor_total_compra'].astype(str).str.replace('.', ",")
  df_cdi['taxa_venda'] = df_cdi['taxa_venda'].astype(str).str.replace('.', ",")
  df_cdi['pu_venda'] = df_cdi['pu_venda'].astype(str).str.replace('.', ",")
  df_cdi['valor_total_venda'] = df_cdi['valor_total_venda'].astype(str).str.replace('.', ",")
  df_cdi['receita_mesa'] = df_cdi['receita_mesa'].astype(str).str.replace('.', ",")
  df_cdi['receita_assessor'] = df_cdi['receita_assessor'].astype(str).str.replace('.', ",")
  df_cdi['taxa_emissao_sem_grossup'] = df_cdi['taxa_emissao_sem_grossup'].astype(str).str.replace('.', ",")
  
  df_di = df_cdi.copy()
  return df_di

In [ ]:
def calculo_pre_venda(df_feriado,df_pre_):
  linhas = len(df_pre_)
  df_pre_['fd_datemissao'] = pd.to_datetime(df_pre_['fd_datemissao'], dayfirst=True)
  df_pre_['Vencimento'] = pd.to_datetime(df_pre_['Vencimento'], dayfirst=True)
  df_pre_['pu_curva'] = np.nan
  #df_pre_['taxa_compra'] = np.nan
  df_pre_['pu_compra'] = np.nan
  df_pre_['valor_total_compra'] = np.nan
  #df_pre_['taxa_venda'] = np.nan
  df_pre_['pu_venda'] = np.nan
  df_pre_['valor_total_venda'] = np.nan
  df_pre_['receita_mesa'] = np.nan
  df_pre_['receita_assessor'] = np.nan
  x=0
  

  for x in range(0,linhas):
    print(x)

    taxa_emissao = df_pre_['taxa_emissao'][x]/100
    quantidade = int(df_pre_['qt_total'][x])
    pu_emissao = float(df_pre_['pu_emissao'][x])
    data_emissao = df_pre_.fd_datemissao[x]
    data_vencimento = df_pre_.Vencimento[x]  

    du_fator = dias_uteis(df_feriado,data_emissao, date.today())
    du_total = dias_uteis(df_feriado,data_emissao,data_vencimento)
    du_restante = dias_uteis(df_feriado,date.today(),data_vencimento)
    print(f'dias uteis restante:{du_restante}')

    fator = ((1+taxa_emissao)**(1/252))**du_fator
    pu_curva = fator * pu_emissao
    df_pre_.at[x,'pu_curva']= pu_curva
    valor_total = pu_curva * quantidade


    #vencimento_date = date(data_vencimento.year,data_vencimento.month,data_vencimento.day)
    print(f'{data_vencimento}')
    dc_vencimento = (date(data_vencimento.year,data_vencimento.month,data_vencimento.day) -date.today()).days
    print(f'pre_vencimento = {dc_vencimento}')
    #pre_vencimento = di_vencimento(dc_vencimento)
    
    taxa_compra = df_pre_['taxa_compra_sem_grossup'][x]/100 
    calculo_compra_1 = (((1+taxa_emissao)**(1/252))**du_total)
    calculo_compra_2 = (((1+taxa_compra)**(1/252))**du_restante)
    calculo_compra = round(calculo_compra_1/calculo_compra_2,8)*pu_emissao
    
    df_pre_.at[x,'pu_compra']= str(calculo_compra)
    df_pre_.at[x,'valor_total_compra']= round(calculo_compra * quantidade,2)
    receita_compra = pu_curva - calculo_compra
    
    taxa_venda = df_pre_['taxa_venda_sem_grossup'][x]/100
    calculo_venda_1 = (((1+taxa_emissao)**(1/252))**du_total)
    calculo_venda_2 = (((1+taxa_venda)**(1/252))**du_restante)
    calculo_venda = round(calculo_venda_1/calculo_venda_2,8)*pu_emissao
    
    df_pre_.at[x,'pu_venda']= str(calculo_venda)
    df_pre_.at[x,'valor_total_venda']= round(calculo_venda * quantidade,2)
    receita_assessor = round((calculo_venda*quantidade)*((1+0.005)**(du_restante/252)-1),4)
    df_pre_.at[x,'receita_mesa']= str((calculo_venda - calculo_compra)*quantidade)
    df_pre_.at[x,'receita_assessor']= str(receita_assessor)


  #print('estou aqui - antes de fazer os replaces')
  df_pre_['taxa_emissao_sem_grossup'] = df_pre_['taxa_emissao_sem_grossup'].astype(str).str.replace('.', ",")
  df_pre_['taxa_emissao'] = df_pre_['taxa_emissao'].astype(str).str.replace('.', ",")
  df_pre_['Vencimento']= pd.to_datetime(df_pre_['Vencimento']).apply(lambda x: x.date())
  df_pre_['fd_datemissao']= pd.to_datetime(df_pre_['fd_datemissao']).apply(lambda x: x.date())
  df_pre_['pu_emissao'] = df_pre_['pu_emissao'].astype(str).str.replace('.', ",")
  df_pre_['pu_curva'] = df_pre_['pu_curva'].astype(str).str.replace('.', ",")
  df_pre_['taxa_compra'] = df_pre_['taxa_compra'].astype(str).str.replace('.', ",")
  df_pre_['pu_compra'] = df_pre_['pu_compra'].astype(str).str.replace('.', ",")
  df_pre_['valor_total_compra'] = df_pre_['valor_total_compra'].astype(str).str.replace('.', ",")
  df_pre_['taxa_venda'] = df_pre_['taxa_venda'].astype(str).str.replace('.', ",")
  df_pre_['pu_venda'] = df_pre_['pu_venda'].astype(str).str.replace('.', ",")
  df_pre_['valor_total_venda'] = df_pre_['valor_total_venda'].astype(str).str.replace('.', ",")
  df_pre_['receita_mesa'] = df_pre_['receita_mesa'].astype(str).str.replace('.', ",")
  df_pre_['receita_assessor'] = df_pre_['receita_assessor'].astype(str).str.replace('.', ",")
  
  df_pre= df_pre_
  return df_pre

In [ ]:
def calculo_ipca_venda(df_ipca_vna,df_feriados,df_ipca_):
  linhas = len(df_ipca_)
  #df_ipca_['fd_datemissao'] = pd.to_datetime(df_ipca_['fd_datemissao'], dayfirst=True)
  #df_ipca_['Vencimento'] = pd.to_datetime(df_ipca_['Vencimento'], dayfirst=True)
  df_ipca_['fd_datemissao'] = pd.to_datetime(df_ipca_['fd_datemissao'], dayfirst=True)
  df_ipca_['fd_datemissao']= pd.to_datetime(df_ipca_['fd_datemissao']).apply(lambda x: x.date())
  df_ipca_['Vencimento'] = pd.to_datetime(df_ipca_['Vencimento'], dayfirst=True)
  df_ipca_['Vencimento']= pd.to_datetime(df_ipca_['Vencimento']).apply(lambda x: x.date())
  df_ipca_['fd_datemissao']= pd.to_datetime(df_ipca_['fd_datemissao']).apply(lambda x: x.date())
  df_ipca_['pu_curva'] = np.nan
  #df_ipca_['taxa_compra'] = np.nan
  df_ipca_['pu_compra'] = np.nan
  df_ipca_['valor_total_compra'] = np.nan
  #df_ipca_['taxa_venda'] = np.nan
  df_ipca_['pu_venda'] = np.nan
  df_ipca_['valor_total_venda'] = np.nan
  df_ipca_['receita_mesa'] = np.nan
  df_ipca_['receita_assessor'] = np.nan
  x=0
  


  for x in range(0,linhas):
    print(x)


    taxa_emissao = df_ipca_['taxa_emissao'][x]/100
    quantidade = int(df_ipca_['qt_total'][x])
    pu_emissao = float(df_ipca_['pu_emissao'][x])
    data_emissao = df_ipca_.fd_datemissao[x]
    data_vencimento = df_ipca_.Vencimento[x] 

    print(f'data_vencimento1 {data_vencimento}')
    print('estou chamando o vna')
    fator_vna = acumulo_ipca_venda(df_ipca_vna,df_feriados,df_ipca_,x)
    print(f'data_vencimento1 {data_vencimento}')
    print(f'voltei da função resposta:  {fator_vna}')


    pu_curva = fator_vna * pu_emissao
    df_ipca_.at[x,'pu_curva']= pu_curva
    valor_total = pu_curva * quantidade
    du_total = dias_uteis(df_feriados,data_emissao,data_vencimento)
    du_restante = dias_uteis(df_feriados, date.today(),data_vencimento)
    data_emissao = df_ipca_.fd_datemissao[x]
    data_vencimento = df_ipca_.Vencimento[x]

    dc_vencimento = (date(data_vencimento.year,data_vencimento.month,data_vencimento.day) -date.today()).days

    print(f'vencimento = {dc_vencimento}')


    #pre_vencimento = di_vencimento(dc_vencimento)
    #implicita_vencimento = inflacao_implicita(dc_vencimento)


    taxa_compra = df_ipca_['taxa_compra_sem_grossup'][x]/100 
    print(f'comprando a {taxa_compra}')
    print(f'meu valor de x é {x}')



    calculo_compra_1 = pu_curva*((((1+taxa_emissao)**(1/252)))**du_restante)
    calculo_compra_2 = (((1+taxa_compra)**(1/252))**du_restante)
    calculo_compra = round(calculo_compra_1/calculo_compra_2,8)
    df_ipca_.at[x,'pu_compra']= str(calculo_compra)
    df_ipca_.at[x,'valor_total_compra']= round(calculo_compra * quantidade,2)
    receita_compra = pu_curva - calculo_compra
    taxa_venda =   df_ipca_['taxa_venda_sem_grossup'][x]/100

    calculo_venda_1 = pu_curva*((((1+taxa_emissao)**(1/252)))**du_restante)
    calculo_venda_2 = (((1+taxa_venda)**(1/252))**du_restante)
    calculo_venda = round(calculo_venda_1/calculo_venda_2,8)
    df_ipca_.at[x,'pu_venda']= str(calculo_venda)
    df_ipca_.at[x,'valor_total_venda']= round(calculo_venda * quantidade,2)
    receita_assessor = round((calculo_venda*quantidade)*((1+0.005)**(du_restante/252)-1),4)
    df_ipca_.at[x,'receita_mesa']= str((calculo_venda - calculo_compra)*quantidade)
    df_ipca_.at[x,'receita_assessor']= str(receita_assessor)

  print('estou aqui - antes de fazer os replaces')
  df_ipca_['taxa_emissao_sem_grossup'] = df_ipca_['taxa_emissao_sem_grossup'].astype(str).str.replace('.', ",")
  df_ipca_['taxa_emissao'] = df_ipca_['taxa_emissao'].astype(str).str.replace('.', ",")
  df_ipca_['Vencimento']= pd.to_datetime(df_ipca_['Vencimento']).apply(lambda x: x.date())
  df_ipca_['fd_datemissao']= pd.to_datetime(df_ipca_['fd_datemissao']).apply(lambda x: x.date())
  df_ipca_['pu_emissao'] = df_ipca_['pu_emissao'].astype(str).str.replace('.', ",")
  df_ipca_['pu_curva'] = df_ipca_['pu_curva'].astype(str).str.replace('.', ",")
  df_ipca_['taxa_compra'] = df_ipca_['taxa_compra'].astype(str).str.replace('.', ",")
  df_ipca_['pu_compra'] = df_ipca_['pu_compra'].astype(str).str.replace('.', ",")
  df_ipca_['valor_total_compra'] = df_ipca_['valor_total_compra'].astype(str).str.replace('.', ",")
  df_ipca_['taxa_venda'] = df_ipca_['taxa_venda'].astype(str).str.replace('.', ",")
  df_ipca_['pu_venda'] = df_ipca_['pu_venda'].astype(str).str.replace('.', ",")
  df_ipca_['valor_total_venda'] = df_ipca_['valor_total_venda'].astype(str).str.replace('.', ",")
  df_ipca_['receita_mesa'] = df_ipca_['receita_mesa'].astype(str).str.replace('.', ",")
  df_ipca_['receita_assessor'] = df_ipca_['receita_assessor'].astype(str).str.replace('.', ",")

  
  df_ipca__ipca = df_ipca_
  return df_ipca__ipca


In [ ]:
def acumulo_ipca_venda(df_ipca_vna, df_feriados, df,x):
  #sheets = 'Master Proprietaria'
  #print('abrindo sheets vna')
  #sh = gc.open(sheets)
  #df_ipca_vna = get_as_dataframe(sh.worksheet('IPCA'), parse_dates=True, evaluate_formulas=True)
  #df_ipca_vna=df_ipca_vna.dropna(how='all')
  #df_ipca_vna=df_ipca_vna.dropna(axis=1, how='all')
  #df_ipca_vna['DATA'] = pd.to_datetime(df_ipca_vna['DATA'], dayfirst=True)
  #df_ipca_vna['DATA'] = pd.to_datetime(df_ipca_vna['DATA']).apply(lambda x: x.date())
  print('meu x é' , x)

  taxa_emissao = float(df['taxa_emissao'][x])/100
  quantidade = int(df['qt_total'][x])
  pu_emissao = int(df['pu_emissao'][x])
  data_emissao = df.fd_datemissao[x]
  data_vencimento = df.Vencimento[x]
  du_total = dias_uteis(df_feriados, data_emissao,data_vencimento)
  du_restante = dias_uteis(df_feriados, date.today(),data_vencimento)
  

  a_year = dateutil.relativedelta.relativedelta(years=1)
  a_month = dateutil.relativedelta.relativedelta(months=1)
  #print(f'A data de Aniversario eh:{data_aniversario}')
  data_aniversario = str(df.fd_datemissao[x])
  data_aniversario = datetime.datetime.strptime(data_aniversario, "%Y-%m-%d")
  print(f'A data de Aniversario eh:{data_aniversario}')
  print('criando datas')
  if data_aniversario.day >= 15:
    data_aniversario_real = date(data_aniversario.year,data_aniversario.month,data_aniversario.day)
    data_aniversario_ajustado = date(data_aniversario.year,data_aniversario.month,15)
  elif data_emissao.day < 15:
    data_aniversario = data_aniversario - a_month
    data_aniversario_real = date(data_aniversario.year,data_aniversario.month,data_aniversario.day)
    data_aniversario_ajustado = date(data_aniversario.year,data_aniversario.month,15)

  print(data_aniversario.day)
  lista_meses_30 = [4,6,9,11]
  lista_fev = [29,30,31]
  if date.today().month in lista_meses_30 and data_aniversario.day == 31:
    data_aniversario_dia = 30
  elif date.today().month == 2 and data_aniversario.day in lista_fev:
    data_aniversario_dia = 28
  else:
    data_aniversario_dia = data_aniversario.day
  data_fim_mes = str(date(date.today().year,date.today().month,data_aniversario_dia))
  data_fim_mes = datetime.datetime.strptime(data_fim_mes, "%Y-%m-%d")

  if data_fim_mes.day > (date.today()).day :
    
    data_fim_mes_real = date(data_fim_mes.year,data_fim_mes.month,data_fim_mes.day)
    if data_fim_mes.day >= 15  and data_fim_mes.year == date.today().year:
      data_fim_mes_ajustado = date(data_fim_mes.year,data_fim_mes.month,15)
    elif data_fim_mes.day < 15 and data_fim_mes.year == date.today().year: 
      data_fim_mes_ajustado = date(data_fim_mes.year,(data_fim_mes-a_month).month,15)
    #elif data_fim_mes.day < 15  and data_fim_mes.year != date.today().year: 
     # data_fim_mes_ajustado = date(date.today().year,(data_fim_mes-a_month).month,15)
    #elif data_fim_mes.day >= 15  and data_fim_mes.year != date.today().year:  
     # data_fim_mes_ajustado = date(date.today().year,data_fim_mes.month,15) 
  elif data_fim_mes.day <= (date.today()).day:

   
    data_fim_mes = data_fim_mes + a_month
    print(f'a data fim_mes{data_fim_mes}')


    data_fim_mes_real = date(data_fim_mes.year,data_fim_mes.month,data_fim_mes.day)
    if data_fim_mes.day >= 15  and data_fim_mes.year == date.today().year:
      data_fim_mes_ajustado = date(data_fim_mes.year,data_fim_mes.month,15)
    elif data_fim_mes.day < 15  and data_fim_mes.year == date.today().year:  
      data_fim_mes_ajustado = date(data_fim_mes.year,(data_fim_mes-a_month).month,15)
    elif data_fim_mes.day < 15  and data_fim_mes.year != date.today().year:  
      data_fim_mes_ajustado = date(date.today().year,(data_fim_mes-a_month).month,15)
    elif data_fim_mes.day >= 15 and data_fim_mes.year != date.today().year:  
      data_fim_mes_ajustado = date(date.today().year,data_fim_mes.month,15)      





  # Verifica se a data de aniversário final é maior que a data atual
  #if data_fim_mes_ajustado > date.today():
    # Se a data de aniversário final for maior, subtrai um ano da data
   # data_fim_mes_ajustado = data_fim_mes_ajustado.replace(year=data_fim_mes_ajustado.year-1)
   # data_fim_mes_real = data_fim_mes_real.replace(year=data_fim_mes_real.year-1)



  data_inicio_mes_real = str(data_fim_mes_real)

  data_inicio_mes_ajustado = str(data_fim_mes_ajustado)


  data_inicio_mes_real = datetime.datetime.strptime(data_inicio_mes_real, "%Y-%m-%d")
  data_inicio_mes_ajustado = datetime.datetime.strptime(data_inicio_mes_ajustado, "%Y-%m-%d")

  data_inicio_mes_real = data_fim_mes_real - a_month
  data_inicio_mes_ajustado = data_fim_mes_ajustado - a_month


  print(data_inicio_mes_ajustado)
  dia_uteis_mes = dias_uteis(df_feriados,data_inicio_mes_real, date.today())-1
  dias_uteis_mes_total = dias_uteis(df_feriados,data_inicio_mes_real,data_fim_mes_real)-1


  print('criando ipca')
  print(data_fim_mes_ajustado)

  


  data_inicio_mes_ipca = df_ipca_vna.query('DATA == @data_inicio_mes_ajustado').reset_index().iat[0,2]
  data_fim_mes_ipca = df_ipca_vna.query('DATA == @data_fim_mes_ajustado').reset_index().iat[0,2]
  projecao_ipca = round((data_fim_mes_ipca/data_inicio_mes_ipca)-1,8)


  ipca_periodo_base = df_ipca_vna.query('DATA == @data_aniversario_ajustado').reset_index().iat[0,2]
  ipca_periodo_atual = data_inicio_mes_ipca

  vna_ipca = round(ipca_periodo_atual/ipca_periodo_base,16)
  projecao = round((1+projecao_ipca)**(dia_uteis_mes/dias_uteis_mes_total),16)
  VNA = round(vna_ipca*projecao,16)

  pre = round((1+taxa_emissao)**((du_total-du_restante)/252),16)

  pre_ipca = round(VNA*pre,16)
  print(f'meu vna eh {vna_ipca}')
  print(f'minha projecao eh  eh {projecao_ipca}')
  print(f'meu pre_ipca eh {pre_ipca}')
  return pre_ipca

# Taxa Venda

In [ ]:
def taxa_venda_():
    """Pegar a Melhor taxa considerando Indexador e Prazo.

    Args:
        prazo (int): prazo em dias corridos, ajustado para base x360
        indexador (string): indexador do título:
         
        - Inflação: "IPCA"
        - Pre-Fixado: "PRE"
        - %CDI: "DI
     
    Return: dataframe com as melhores taxas já grossapadas
    """
    #prazo = 360 * round(prazo /360)
    print('estou aqui')
    time.sleep(1)
    #if prazo < 360 and indexador != 'IPCA+': prazo = 180 
    #elif prazo < 360 and indexador == 'IPCA+': prazo = 360
    #indexadores_lista = ['IPCA+','CDI','PRE']
    #print(prazo)
    
    
    print('estou aqui 2')
    sheets = 'PLANILHA DE TAXAS DIÁRIAS'
    sh = gc.open(sheets)
    taxas = gc.open(sheets).worksheet('Melhores_Taxas')
    planilha = pd.DataFrame(taxas.get_all_records())
    planilha = planilha[['Ativo', 'Prazo', 'Indexador', 'Taxa Gross Up']]
    planilha['Taxa Gross Up'] = planilha['Taxa Gross Up'].astype(str).str.replace(',', ".")
    planilha['Taxa Gross Up'] = planilha['Taxa Gross Up'].astype(str).str.replace('%', "")
    planilha['Taxa Gross Up'] = planilha['Taxa Gross Up'].astype(float)
    planilha['Taxa Gross Up'] = round(planilha['Taxa Gross Up']/100,6)
    

    print('estou aqui 3')
    df_pre = planilha.query('Indexador == "PRE"')
    df_cdi = planilha.query('Indexador == "CDI"')
    df_ipca = planilha.query('Indexador == "IPCA+"')
    df_cdimais = planilha.query('Indexador == "CDI+"')


    df_pre = pd.pivot_table(df_pre, index=['Prazo'], values='Taxa Gross Up', aggfunc=['max']).reset_index()
    df_pre.columns = ['Prazo', 'Taxa']
    df_pre['Indexador'] = 'PRE'

    df_cdi = pd.pivot_table(df_cdi, index=['Prazo'], values='Taxa Gross Up', aggfunc=['max']).reset_index()
    df_cdi.columns = ['Prazo', 'Taxa']
    df_cdi['Indexador'] = 'CDI'

    df_ipca = pd.pivot_table(df_ipca, index=['Prazo'], values='Taxa Gross Up', aggfunc=['max']).reset_index()
    df_ipca.columns = ['Prazo', 'Taxa']
    df_ipca['Indexador'] = 'IPCA+'

    df_cdimais = pd.pivot_table(df_cdimais, index=['Prazo'], values='Taxa Gross Up', aggfunc=['max']).reset_index()
    df_cdimais.columns = ['Prazo', 'Taxa']
    df_cdimais['Indexador'] = 'CDI+'

    dfs = [df_ipca,df_pre,df_cdimais]

    tabela_append = df_cdi.append(df_ipca, ignore_index=True)
    tabela_2_append = tabela_append.append(df_cdimais, ignore_index=True)
    melhores_taxas_primario = tabela_2_append.append(df_pre, ignore_index=True)
    melhores_taxas_primario = melhores_taxas_primario.query('Taxa != 0').reset_index(drop=True)
    melhor_taxa = melhores_taxas_primario
    #melhor_taxa = melhores_taxas_primario.query('Indexador == @indexador & Prazo == @prazo' )
    #melhor_taxa = melhor_taxa['Taxa'].values[0]

        
    return melhor_taxa

melhores_taxas = taxa_venda_()

In [ ]:
melhores_taxas

In [ ]:
teste_pre = melhores_taxas.copy()
teste_pre.query("Prazo != 0")
implicita = [teste_pre,implicita_interpolada]
teste_pre = pd.merge(teste_pre, implicita_interpolada, right_index=True, left_on = 'Prazo', how = 'inner')
teste_pre.reset_index(drop=True,inplace=True)
implicita_forcada = teste_pre.loc[3,'Implicita']
for i in range(0,len(teste_pre.Prazo)):
  if teste_pre.loc[i,'Prazo'] < 60:
    teste_pre.at[i,'Implicita'] = implicita_forcada

teste_pre = pd.merge(teste_pre, pre_interpolado, right_index=True, left_on = 'Prazo', how = 'inner') 
teste_pre.reset_index(drop=True,inplace=True)
#teste_pre = pd.concat(implicita, join='inner', axis=1).fillna(np.nan)
#di_futuro = [teste_pre, pre_interpolado] 
#teste_pre = pd.concat(di_futuro, join='inner', axis=1).fillna(np.nan)
#teste_pre['DI-1'] = round(teste_pre['DI-1'],4)
teste_pre

for i in range(0,len(teste_pre.Indexador)):
  if teste_pre.loc[i,'Indexador'] == "CDI":
    teste_pre.loc[i,'pre_equivalente'] = cdi_to_pre(teste_pre.loc[i,'Taxa'],teste_pre.loc[i,'DI-1'])
  elif teste_pre.loc[i,'Indexador'] == "CDI+":
    teste_pre.loc[i,'pre_equivalente'] = cdi_mais_to_pre(teste_pre.loc[i,'Taxa'],teste_pre.loc[i,'DI-1'])
  elif teste_pre.loc[i,'Indexador'] == "IPCA+":
    teste_pre.loc[i,'pre_equivalente'] = ipca_to_pre(teste_pre.loc[i,'Taxa'],teste_pre.loc[i,'Implicita'])
  elif teste_pre.loc[i,'Indexador'] == "PRE":
    teste_pre.loc[i,'pre_equivalente'] = teste_pre.loc[i,'Taxa']

tta = pd.pivot_table(teste_pre, index=['Prazo','Implicita','DI-1'], values='pre_equivalente', aggfunc=['max']).reset_index()
tta.columns = ['Prazo','Implicita','DI_Futuro', 'Rentabilidade']

for i in range(0,len(tta.Prazo)):
  if tta.loc[i,'Prazo'] <= 1080:

    tta.loc[i,'Rentabilidade'] = tta.loc[i,'Rentabilidade'] + 0.010
  else:
    tta.loc[i,'Rentabilidade'] = tta.loc[i,'Rentabilidade'] + 0.022
tabela_ipca = tta.copy()
tabela_ipca['Indexador'] = 'IPCA+'
tabela_ipca['Taxa'] = np.nan
for i in range(0,len(tabela_ipca.Prazo)):
  tabela_ipca.at[i,'Taxa'] = pre_to_ipca(tabela_ipca.loc[i,'Rentabilidade'],tabela_ipca.loc[i,'Implicita'])




tabela_cdi = tta.copy()
tabela_cdi['Indexador'] = 'CDI'
tabela_cdi['Taxa'] = np.nan
for i in range(0,len(tabela_cdi.Prazo)):
  tabela_cdi.at[i,'Taxa'] = pre_to_cdi(tabela_cdi.loc[i,'Rentabilidade'],tabela_cdi.loc[i,'DI_Futuro'])



tabela_cdi_mais = tta.copy()
tabela_cdi_mais['Indexador'] = 'CDI+'
tabela_cdi_mais['Taxa'] = np.nan
for i in range(0,len(tabela_cdi_mais.Prazo)):
  tabela_cdi_mais.at[i,'Taxa'] = pre_to_cdi_mais(tabela_cdi_mais.loc[i,'Rentabilidade'],tabela_cdi_mais.loc[i,'DI_Futuro'])


tabela_pre = tta.copy()
tabela_pre['Indexador'] = 'PRE'
tabela_pre['Taxa'] = np.nan
for i in range(0,len(tabela_pre.Prazo)):
  tabela_pre.at[i,'Taxa'] = tabela_pre.loc[i,'Rentabilidade']


tabela_ipca = tabela_ipca[['Prazo','Indexador','Taxa']]
tabela_cdi = tabela_cdi[['Prazo','Indexador','Taxa']]
tabela_cdi_mais = tabela_cdi_mais[['Prazo','Indexador','Taxa']]
tabela_pre = tabela_pre[['Prazo','Indexador','Taxa']]



tabela_melhores_taxas = concat(tabela_pre,tabela_ipca, tabela_cdi, tabela_cdi_mais )

tabela_melhores_taxas.reset_index(drop=True,inplace=True)
melhores_taxas_normalizada = tabela_melhores_taxas

t1 = pd.merge(tabela_pre, tabela_ipca , right_on ='Prazo', left_on = 'Prazo', how = 'inner') 
t1 = t1.rename(columns={'Taxa_x':'PRE','Taxa_y':'IPCA+'})
t1.drop(['Indexador_x', 'Indexador_y'], axis=1, inplace=True)
t2 = pd.merge(t1, tabela_cdi, right_on ='Prazo', left_on = 'Prazo', how = 'inner') 
t2 = t2.rename(columns={'Taxa':'%DI'})
t2.drop(['Indexador'], axis=1, inplace=True)
t3 = pd.merge(t2, tabela_cdi_mais, right_on ='Prazo', left_on = 'Prazo', how = 'inner') 
t3 = t3.rename(columns={'Taxa':'DI+'})
t3.drop(['Indexador'], axis=1, inplace=True)
tabela_subir_gs = t3.reset_index(drop=True)
tabela_subir_gs['PRE'] = tabela_subir_gs['PRE'].astype(str).str.replace(".",',')
tabela_subir_gs['IPCA+'] = tabela_subir_gs['IPCA+'].astype(str).str.replace(".",',')
tabela_subir_gs['%DI'] = tabela_subir_gs['%DI'].astype(str).str.replace(".",',')
tabela_subir_gs['DI+'] = tabela_subir_gs['DI+'].astype(str).str.replace(".",',')

sheets = 'PLANILHA DE TAXAS DIÁRIAS'
sh = gc.open(sheets)
propria_posicao_sheets = gc.open(sheets).worksheet('Taxas_Equivalencia')
set_with_dataframe(propria_posicao_sheets, tabela_subir_gs)


In [ ]:
def bq2df(sql):
  """Função para retornar a query como um dataframe"""
  query = client.query(sql)
  results = query.result()
  return results.to_dataframe()



def data_posicao():
  query = """
  SELECT distinct codigo_cetip,nome_emissor, produto,indexador, dt_emissao, dt_vencimento, taxa, pu_emissao  FROM XXXXXXXXXXX
  """
  catalogo_posicao = bq2df(query)
  
  return catalogo_posicao




# Apertar Ctrl+F8 a partir da linha abaixo!

In [ ]:
catalogo = data_posicao()
catalogo

# Secundário

In [ ]:
url = 'secundario_120423.json'

df = pd.read_json(url, encoding='latin-1')

dados = []
df_secundario = pd.DataFrame(dados, columns=["CBLC","Instrumento Financeiro","Emissor","Ativo",
                                             "Indexador","Taxa Emissao","Emissao","Vencimento","PU Antecipado",
                                            "Quantidade"])
data = df['BondAssetList']
for i in range(0,len(data)):   
    df_temp = df['BondAssetList'][i]
    df_child_temp = pd.DataFrame (df_temp)
    for child in range(0,len(df_child_temp)):
        df_granchild_temp = df_child_temp['BondOrderList'][child]
        df_grandchild = pd.DataFrame(df_granchild_temp)
        for grandchild in range(0,len(df_grandchild)):
            lista_temp = []
            cblc = df_grandchild['CBLCId'][grandchild]
            cod_cetip = df_grandchild['InstrumentId'][grandchild]
            emissor = df_grandchild['IssuerName'][grandchild]
            ativo = df_grandchild['BondAsset'][grandchild]
            indexador = df_grandchild['Indexer'][grandchild]
            taxa = df_grandchild['RateReturn'][grandchild]
            dt_emissao = df_grandchild['EmissionDate'][grandchild]
            dt_vencimento = df_grandchild['MaturityDate'][grandchild]
            pu_antecipado = df_grandchild['AnticipateUnitPrice'][grandchild]
            qtd = df_grandchild['Amount'][grandchild]

            lista_temp.extend([cblc,cod_cetip,emissor,ativo,indexador,taxa,dt_emissao,dt_vencimento,pu_antecipado,qtd])
            df_secundario.loc[len(df_secundario)] = lista_temp
            




df_secundario = df_secundario.query("Emissor != 'Dacasa Financeira'")
df_secundario = df_secundario.query("Emissor != 'BRK Financeira'")
df_secundario['PU Antecipado'] = round(df_secundario['PU Antecipado'],8)
df_secundario.reset_index(drop=True,inplace=True)

change_name = {'codigo_cetip':'Instrumento Financeiro'}
change_name_new_df = {'dt_emissao':'Emissao','dt_vencimento':'Vencimento','taxa':'Taxa Emissao'}

catalogo_ajustado = catalogo[['codigo_cetip', 'dt_emissao','dt_vencimento','taxa']].copy()
catalogo_ajustado.rename(columns=change_name,inplace=True)


df_secundario = df_secundario.merge(catalogo_ajustado, on='Instrumento Financeiro', how='left')
df_secundario.drop(['Taxa Emissao', 'Emissao','Vencimento'], axis=1, inplace=True)
df_secundario.rename(columns=change_name_new_df,inplace=True)
df_secundario = df_secundario.dropna()
df_secundario = df_secundario.reset_index()

In [ ]:
df_secundario

In [ ]:
""""Ajustando Dados"""
"Limpando Ativo"
df_secundario.Ativo = df_secundario['Ativo'].str[:3].str.strip()
"Ajustando Data"
x=0

"""
for emissao in df_secundario.Emissao:
    df_secundario.at[x,'Emissao'] = datechange(emissao)
    df_secundario.at[x,'Emissao'] = datetime.datetime.strptime(df_secundario.loc[x,'Emissao'], '%d/%m/%Y')
    df_secundario.at[x,'Vencimento'] = datechange(str(df_secundario.loc[x,"Vencimento"]))
    df_secundario.at[x,'Vencimento'] = datetime.datetime.strptime(df_secundario.loc[x,'Vencimento'], '%d/%m/%Y')
    x= x+1
"""
 
for i in range(0,len(df_secundario['PU Antecipado'])):
    check_pu = df_secundario.loc[i,'PU Antecipado']/1000
    if check_pu < 0.7 and check_pu > 0.0001:
        df_secundario.at[i,'PU Emissao'] = 1
    elif check_pu > 0.7:
               df_secundario.at[i,'PU Emissao'] = 1000
    else:
        df_secundario.at[i,'PU Emissao'] = 0.01

        
df_secundario['volume_hoje'] = df_secundario.Quantidade * df_secundario['PU Antecipado']
df_secundario['Assessor'] = "Secundário"


df_secundario['Chave Unica'] = df_secundario.CBLC + df_secundario.Emissor + df_secundario.Indexador

dict = {'CBLC': 'cblc',
        'Ativo': 'Produto',
        'Indexador': 'fc_indice',
        'Instrumento Financeiro':'fc_minemonico',
        'Taxa Emissao': 'taxa_emissao',
        'Emissao': 'fd_datemissao',
        'Emissor': 'Nome_Emissor',
        'Quantidade': 'qt_total',
        'PU Emissao': 'pu_emissao'

       }
df_secundario.rename(columns=dict,inplace=True)       
        
#df_secundario['taxa_emissao'] = df_secundario['taxa_emissao'].astype(str).str.replace(".",",")

df_secundario['dias_corridos'] = 1

for dias in range(0,len(df_secundario.dias_corridos)):
    ano = df_secundario.Vencimento[dias].year
    mes = df_secundario.Vencimento[dias].month
    dia = df_secundario.Vencimento[dias].day

    df_secundario.at[dias,'dias_corridos'] = (date(ano,mes,dia) -date.today()).days


In [ ]:
plan_cliente

In [ ]:
""
#plan_cliente = files.upload()
#plan_cliente = pd.read_excel('secundario_180822.xlsx')
plan_cliente = df_secundario
#melhores_taxas = taxa_venda_()
#plan_cliente = plan_cliente.query('volume_hoje >= 30000')
#plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('CDI','DI')
plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('IPCA','IPCA+')


plan_cliente.melhor_taxa = np.nan
plan_cliente.melhor_taxa_venda = np.nan
plan_cliente.taxa_venda = np.nan
plan_cliente.melhor_taxa_normalizada = np.nan
plan_cliente.taxa_compra = np.nan
plan_cliente.taxa_emissao_normalizada = np.nan

for i in range(0,len(plan_cliente.fc_indice)):
  indexador = plan_cliente.loc[i,'fc_indice']
  prazo = plan_cliente.loc[i,'dias_corridos']
  ativo = plan_cliente.loc[i,'Produto']
  dcs = plan_cliente.loc[i,'dias_corridos']
  prazo = 360 * round(prazo /360)
  if prazo < 30 : prazo = 30 
  #elif prazo < 360 and indexador == 'IPCA+': prazo = 360
  melhor_taxa = melhores_taxas_normalizada.query('Indexador == @indexador & Prazo == @prazo')

  valor = melhor_taxa['Taxa'].values[0]
  plan_cliente.loc[i,'melhor_taxa'] = round(valor*100,6)
  #melhor_taxa = plan_cliente.loc[i,'melhor_taxa']


  if prazo < 360 and indexador !=	'IPCA+': 
      prazo_venda = 180 
  elif prazo < 360 and indexador == 'IPCA+': 
      prazo_venda = 360
  else:
    prazo_venda = prazo

  melhor_taxa_venda_query = melhores_taxas.query('Indexador == @indexador & Prazo == @prazo_venda')
  melhor_taxa_venda = melhor_taxa_venda_query['Taxa'].values[0]
  plan_cliente.loc[i,'melhor_taxa_venda_hub'] = round(melhor_taxa_venda*100,6)
  #melhor_taxa_venda_produto = plan_cliente.loc[i,'melhor_taxa_venda_hub']

x=0
lista_ativo = ['LCA','LCI']
for i in plan_cliente.fc_indice:
  ativo = plan_cliente.loc[x,'Produto']
  dcs = float(plan_cliente.loc[x,'dias_corridos'])
  melhor_taxa = plan_cliente.loc[x,'melhor_taxa']
  melhor_taxa_venda_produto = plan_cliente.loc[x,'melhor_taxa_venda_hub']
  taxa_cliente = plan_cliente.loc[x,'taxa_emissao']
  
  nao_CDI = ['CDI','CDI+']
  if i not in nao_CDI: 
    ajuste_compra = 4.5 
    ajuste_venda = 0.75
  elif i == "CDI": 
    ajuste_compra = 6 
    ajuste_venda = 2.5
  elif i == "CDI+":
    ajuste_compra = 1.5
    ajuste_venda = 0.50

  plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa) + float(ajuste_compra),6)
  plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)

  if plan_cliente.loc[x,'taxa_venda'] > melhor_taxa and i not in nao_CDI:
    plan_cliente.loc[x,'taxa_venda'] = round(melhor_taxa +0.25,6)
  elif plan_cliente.loc[x,'taxa_venda'] > melhor_taxa and i in nao_CDI:
    plan_cliente.loc[x,'taxa_venda'] = round(melhor_taxa +1,6)


  if ativo in lista_ativo:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_compra'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_venda'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_emissao'],indexador)*100,6)
  else:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(plan_cliente.loc[x,'taxa_compra'],6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(plan_cliente.loc[x,'taxa_venda'],6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(plan_cliente.loc[x,'taxa_emissao'],6)

  x= x+1

#plan_cliente.drop(['melhor_taxa_venda'], axis=1)

temp_pre_cliente = None
temp_cdi_cliente = None
temp_ipca_cliente = None
plan_cliente_cdi = plan_cliente.query('fc_indice == "CDI"').copy().reset_index(drop=True)
if len(plan_cliente_cdi) > 0:
  plan_cliente_cdi = calculo_cdi_venda(pre_interpolado,fer,plan_cliente_cdi)
plan_cliente_pre = plan_cliente.query('fc_indice == "PRE"').copy().reset_index(drop=True)
if len(plan_cliente_pre) > 0:
  plan_cliente_pre = calculo_pre_venda(fer, plan_cliente_pre)
plan_cliente_ipca = plan_cliente.query('fc_indice == "IPCA+"').copy().reset_index(drop=True)
if len(plan_cliente_ipca) > 0:
  plan_cliente_ipca = calculo_ipca_venda(df_ipca_vna,fer,plan_cliente_ipca)
  
  
df_clientes_final = concat(*[plan_cliente_cdi, plan_cliente_pre, plan_cliente_ipca])
df_clientes_final = df_clientes_final.reset_index(drop=True).reset_index(drop=True)
df_clientes_final.rename(columns = {'receita_mesa':'receita_total'}, inplace = True)





df_secundario_input = df_clientes_final.filter(['fc_minemonico','Nome_Emissor','Produto', 'fc_indice','qt_total','Vencimento'
                                                , 'dias_corridos','taxa_venda_sem_grossup','pu_venda','valor_total_venda','receita_assessor' ], axis=1)
df_secundario_input['ROA'] = np.nan
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(str).str.replace(',','.')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(str).str.replace(',','.')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(float)
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(float)
df_secundario_input['ROA'] =  round(df_secundario_input.receita_assessor/ df_secundario_input.valor_total_venda,4)
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(str).str.replace('.',',')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(str).str.replace('.',',')
df_secundario_input['ROA'] = df_secundario_input['ROA'].astype(str).str.replace('.',',')

df_secundario_input = df_secundario_input.drop('receita_assessor',axis=1)

df_secundario_input.columns = ['Instrumento Financeiro','Emissor','produto','indexador','Quantidade Inicial','Vencimento','Dias Corridos - Vencimento','Taxa Negociação','Pu negociação','Valor Financeiro','ROA']


#df_clientes_final.to_excel("df_clientes_final_sec.xlsx")
#files.download('df_clientes_final_sec.xlsx')


#df_secundario_input.to_excel("secundario_input.xlsx")
#files.download('secundario_input.xlsx')


#dfs = [df_clientes_final,df_secundario_input]
#sheets = ['lista_completa', 'layout_heimdall']


#dfs_tabs(dfs, sheets, f'secundario_heimdall_{datetime.datetime.now()}.xlsx')



In [ ]:
df_clientes_final.pu_compra = df_clientes_final.pu_compra.astype(str).str.replace(',','.')
df_clientes_final.pu_venda = df_clientes_final.pu_venda.astype(str).str.replace(',','.')
df_clientes_final.receita_assessor = df_clientes_final.receita_assessor.astype(str).str.replace(',','.')
df_clientes_final['PU Antecipado'] = df_clientes_final['PU Antecipado'].astype(str).str.replace(',','.')
for i in range(0,len(df_clientes_final)):
  nova_receita = round(float(df_clientes_final.loc[i,'pu_venda']) - float(df_clientes_final.loc[i,'PU Antecipado']),6)
  nova_receita = round(nova_receita*int(df_clientes_final.loc[i,'qt_total']),6)
  df_clientes_final.at[i,'receita_total'] = nova_receita
  df_clientes_final.at[i,'Receita Mesa'] = nova_receita - float(df_clientes_final.loc[i,'receita_assessor'])
  if float(nova_receita) < float(df_clientes_final.loc[i,'receita_assessor']):
    df_clientes_final.at[i,'Check_Receita_Assessor'] = "NOK"
  else:
    df_clientes_final.at[i,'Check_Receita_Assessor'] = "Ok"

for i in range(0,len(df_clientes_final.pu_compra)):
  check_pu = float(df_clientes_final.loc[i,'pu_compra'])/float(df_clientes_final.loc[i,'PU Antecipado'])
  if float(df_clientes_final.loc[i,'pu_venda']) < float(df_clientes_final.loc[i,'PU Antecipado']) : 
    check_pu_venda = "NOK"
  else:
    check_pu_venda = "OK"
  if check_pu < 0.9 or check_pu_venda == "NOK" :

    df_clientes_final.at[i,'Check'] = "NOK"
  else:
    df_clientes_final.at[i,'Check'] = 'OK'

df_clientes_final.drop(df_clientes_final[df_clientes_final.Check == 'NOK'].index, inplace=True)
df_clientes_final.reset_index(drop=True, inplace = True)
df_clientes_final.drop(df_clientes_final[df_clientes_final['Check_Receita_Assessor'] == 'NOK'].index, inplace=True)
df_clientes_final.reset_index(drop=True, inplace = True)

df_clientes_final.pu_venda = df_clientes_final.pu_venda.astype(str).str.replace('.',',')
df_clientes_final.pu_compra = df_clientes_final.pu_compra.astype(str).str.replace('.',',')
df_clientes_final['PU Antecipado'] = df_clientes_final['PU Antecipado'].astype(str).str.replace('.',',')
df_clientes_final.receita_assessor = df_clientes_final.receita_assessor.astype(str).str.replace('.',',')

for i in range(0,len(df_clientes_final.Vencimento)):
  ano = df_clientes_final.Vencimento[i].year
  ano = str(ano)
  mes = df_clientes_final.Vencimento[i].month
  if mes < 10: 
    mes = str('0'+str(mes))
  else:
    mes = str(mes)

  dia = df_clientes_final.Vencimento[i].day
  if dia < 10: 
    dia = str('0'+str(dia))
  else:
    dia = str(dia)

  df_clientes_final.at[i,'Vencimento'] = str(dia+'/'+mes+'/'+ano)


df_clientes_final['Chave Unica'] = df_clientes_final['Chave Unica'] + df_clientes_final.Vencimento.astype(str) + df_clientes_final['PU Antecipado']


In [ ]:
#@title exportar excel - secundário
df_clientes_xls = df_clientes_final
df_clientes_xls.to_excel(f'secundario_extranet{datetime.datetime.now()}.xlsx')


In [ ]:
df_clientes_xls

In [ ]:
df_clientes_final

In [ ]:
plan_cliente = plan_cliente.query('dias_corridos > 30').reset_index(drop=True)
plan_cliente

In [ ]:
#plan_cliente = files.upload()
#plan_cliente = pd.read_excel('assessoria_santander2610.xlsx')
#plan_cliente = pd.read_excel('pricing_ararinha_2410.xlsx')
#plan_cliente = plan_cliente.query('fc_indice != "CDI"')
#plan_cliente.reset_index(drop=True,inplace=True)
#melhores_taxas = taxa_venda_()
#plan_cliente = plan_cliente.query('volume_hoje >= 10000')
plan_cliente.reset_index(drop=True ,inplace=True)
print(len(plan_cliente))
#plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('CDI','DI')
plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('IPCA','IPCA+')

for i in range(0,len(plan_cliente.dias_corridos)):
  print(i)
  ano = plan_cliente.Vencimento[i].year
  mes = plan_cliente.Vencimento[i].month
  dia = plan_cliente.Vencimento[i].day

  plan_cliente.at[i,'dias_corridos'] = (date(ano,mes,dia) -date.today()).days



plan_cliente = plan_cliente.query('dias_corridos > 30').reset_index(drop=True)
plan_cliente.melhor_taxa = np.nan
plan_cliente.melhor_taxa_venda = np.nan
plan_cliente.taxa_venda = np.nan
plan_cliente.melhor_taxa_normalizada = np.nan
plan_cliente.taxa_compra = np.nan
plan_cliente.taxa_emissao_normalizada = np.nan

for i in range(0,len(plan_cliente.fc_indice)):
  indexador = plan_cliente.loc[i,'fc_indice']
  prazo = plan_cliente.loc[i,'dias_corridos']
  ativo = plan_cliente.loc[i,'Produto']
  dcs = plan_cliente.loc[i,'dias_corridos']
  prazo = 360 * round(prazo /360)
  if prazo < 30 : prazo = 30 
  #elif prazo < 360 and indexador == 'IPCA+': prazo = 360
  melhor_taxa = melhores_taxas_normalizada.query('Indexador == @indexador & Prazo == @prazo')

  valor = melhor_taxa['Taxa'].values[0]
  plan_cliente.loc[i,'melhor_taxa'] = round(valor*100,6)
  #melhor_taxa = plan_cliente.loc[i,'melhor_taxa']


  if prazo < 360 and indexador !=	'IPCA+': 
      prazo_venda = 180 
  elif prazo < 360 and indexador == 'IPCA+': 
      prazo_venda = 360
  else:
    prazo_venda = prazo

  melhor_taxa_venda_query = melhores_taxas.query('Indexador == @indexador & Prazo == @prazo_venda')
  melhor_taxa_venda = melhor_taxa_venda_query['Taxa'].values[0]*100
  plan_cliente.loc[i,'melhor_taxa_venda'] = round(melhor_taxa_venda,6)
  #melhor_taxa_venda_produto = plan_cliente.loc[i,'melhor_taxa_venda']

x=0
lista_ativo = ['LCA','LCI']
for i in plan_cliente.fc_indice:
  ativo = plan_cliente.loc[x,'Produto']
  dcs = float(plan_cliente.loc[x,'dias_corridos'])
  melhor_taxa = plan_cliente.loc[x,'melhor_taxa']
  melhor_taxa_venda_produto = plan_cliente.loc[x,'melhor_taxa_venda']
  taxa_cliente = plan_cliente.loc[x,'taxa_emissao']
  if i != 'CDI': 
    ajuste_compra = 7 
    ajuste_venda = 0.75
  else: 
    ajuste_compra = 14 
    ajuste_venda = 2.5
  


  if melhor_taxa <= melhor_taxa_venda_produto:
    
    plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa_venda_produto) + float(ajuste_compra),6)
    plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)

  else:
    plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa) + float(ajuste_compra),6)
    plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)


  if ativo in lista_ativo:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_compra'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_venda'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_emissao'],indexador)*100,6)
  else:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(plan_cliente.loc[x,'taxa_compra'],6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(plan_cliente.loc[x,'taxa_venda'],6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(plan_cliente.loc[x,'taxa_emissao'],6)

  x= x+1

plan_cliente.drop(['melhor_taxa_venda'], axis=1)

temp_pre_cliente = None
temp_cdi_cliente = None
temp_ipca_cliente = None
plan_cliente_cdi = plan_cliente.query('fc_indice == "CDI"').copy().reset_index(drop=True)
if len(plan_cliente_cdi) > 0:
  plan_cliente_cdi = calculo_cdi_venda(pre_interpolado,fer,plan_cliente_cdi)
plan_cliente_pre = plan_cliente.query('fc_indice == "PRE"').copy().reset_index(drop=True)
if len(plan_cliente_pre) > 0:
  plan_cliente_pre = calculo_pre_venda(fer, plan_cliente_pre)

# Assessor

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'XXXXXXXXXX' # Project ID inserted based on the query results selected to explore
location = 'XXXXXXXXXXXXX' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()
job = client.get_job('XXXXXXXXXXXXXXXXXX') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()

In [ ]:
results

In [ ]:
resultados = results.query('assessor_ajustado != "Sem assessor"')

In [ ]:
resultados

In [ ]:
# @title Ajuste de posição
posicao = resultados[resultados["anomes"] == "202303"]
import datetime
import pandas as pd
import numpy as np


# Converte a coluna fd_datvencimento em formato de data
posicao["fd_datvencimento"] = pd.to_datetime(posicao["fd_datvencimento"], format="%Y-%m-%d")

# Pega a data de hoje
hoje = np.datetime64(datetime.datetime.today().date())

# Filtre a tabela para obter apenas as linhas com fd_datvencimento maior que a data de hoje
posicao = posicao[posicao["fd_datvencimento"] > hoje]


# Filtre a tabela para obter apenas as linhas com assessor_ajustado diferente de "Sem Assessor" e "Felipe Borges"
posicao = posicao[~posicao["assessor_ajustado"].isin(["Sem Assessor","Sem assessor", "Felipe Borges"])]

# Filtre a tabela para obter apenas as linhas com tipo_produto igual a "Bancário"
posicao = posicao[posicao["tipo_produto"] == "Bancário"]


# Remove as colunas especificadas
posicao = posicao.drop(["Proprietario", "Assessor", "Max_data_ref", "Right_Max_data_ref"], axis=1)

# Filtre a tabela para obter apenas as linhas com Volume_investido_disponivel maior que 1000
posicao = posicao[posicao["Volume_investido_disponivel"] > 0]
# Reordena as colunas da tabela


colunas_reordenadas = ["data_ref", "cblc", "assessor_ajustado", "fc_minemonico", "quantidade", "fn_pucurva"]
posi_temp = posicao[colunas_reordenadas]

posi_temp = posi_temp.rename(columns={"fc_minemonico": "codigo_cetip"})


In [ ]:
#@title Criando novo dataframe
posicao_final = pd.merge(posi_temp, catalogo, on="codigo_cetip", how="left")
posicao_final.dropna(inplace=True)
posicao_final["concat"] = posicao_final["codigo_cetip"]
# Converte a coluna "dt_vencimento" para o tipo "pd.Timestamp"
posicao_final["dt_vencimento"] = pd.to_datetime(posicao_final["dt_vencimento"])

# Calcula a subtração entre a coluna "dt_vencimento" e a data de hoje
hoje = pd.Timestamp(datetime.datetime.today().date())
posicao_final["dias_corridos"] = (posicao_final["dt_vencimento"] - hoje).dt.days

posicao_final["dt_vencimento"] = posicao_final["dt_vencimento"].dt.date

posicao_final["volume_hoje"] = round(posicao_final["fn_pucurva"]*posicao_final["quantidade"],2)





# Mapeamento dos novos nomes das colunas
nomes_colunas = {"cblc": "CBLC",
                 "produto": "Produto",
                 "indexador": "fc_indice",
                 "codigo_cetip": "fc_minemonico",
                 "dt_emissao": "fd_datemissao",
                 "dt_vencimento": "Vencimento",
                 "nome_emissor": "Nome_Emissor",
                 "quantidade": "qt_total",
                 "assessor_ajustado":"Assessor",
                 "taxa":'taxa_emissao'}

# Troca os nomes das colunas usando o método "rename"
posicao_final = posicao_final.rename(columns=nomes_colunas)

colunas = ['data_ref','fn_pucurva','CBLC', 'Produto', 'fc_indice', 'fc_minemonico', 'concat', 'fd_datemissao', 'Vencimento', 'dias_corridos', 'taxa_emissao', 'Nome_Emissor', 'qt_total', 'volume_hoje', 'pu_emissao', 'Assessor']

posicao_precificacao = posicao_final[colunas]
posicao_precificacao = posicao_precificacao[posicao_precificacao["Nome_Emissor"] != "DACASA FINANCEIRA"]
posicao_precificacao = posicao_precificacao[posicao_precificacao["fc_indice"] != "CDI+"]

posicao_precificacao.to_excel('posicao_preci.xlsx')


#posi_amostral = posicao_precificacao.sample(n=3000, random_state=42)

In [ ]:
posicao_precificacao

In [ ]:
#@title precificando
#plan_cliente = files.upload()
plan_cliente = posicao_precificacao
#plan_cliente = pd.read_excel('141122_pricing_ararinha.xlsx')
#plan_cliente = plan_cliente.query('fc_indice != "CDI"')
#plan_cliente.reset_index(drop=True,inplace=True)
#melhores_taxas = taxa_venda_()
#plan_cliente = plan_cliente.query('volume_hoje >= 10000')
plan_cliente.reset_index(drop=True ,inplace=True)
print(len(plan_cliente))
#plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('CDI','DI')
plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('IPCA','IPCA+')

#for i in range(0,len(plan_cliente.dias_corridos)):
 # print(i)
 # ano = plan_cliente.Vencimento[i].year
 # mes = plan_cliente.Vencimento[i].month
 # dia = plan_cliente.Vencimento[i].day

  #plan_cliente.at[i,'dias_corridos'] = (date(ano,mes,dia) -date.today()).days



plan_cliente = plan_cliente.query('dias_corridos > 0').reset_index(drop=True)
plan_cliente.melhor_taxa = np.nan
plan_cliente.melhor_taxa_venda = np.nan
plan_cliente.taxa_venda = np.nan
plan_cliente.melhor_taxa_normalizada = np.nan
plan_cliente.taxa_compra = np.nan
plan_cliente.taxa_emissao_normalizada = np.nan

for i in range(0,len(plan_cliente.fc_indice)):
  indexador = plan_cliente.loc[i,'fc_indice']
  prazo = plan_cliente.loc[i,'dias_corridos']
  ativo = plan_cliente.loc[i,'Produto']
  dcs = plan_cliente.loc[i,'dias_corridos']
  prazo = 360 * round(prazo /360)
  if prazo < 30 : prazo = 30 
  #elif prazo < 360 and indexador == 'IPCA+': prazo = 360
  melhor_taxa = melhores_taxas_normalizada.query('Indexador == @indexador & Prazo == @prazo')

  valor = melhor_taxa['Taxa'].values[0]
  plan_cliente.loc[i,'melhor_taxa'] = round(valor*100,6)
  #melhor_taxa = plan_cliente.loc[i,'melhor_taxa']


  if prazo < 360 and indexador !=	'IPCA+': 
      prazo_venda = 180 
  elif prazo < 360 and indexador == 'IPCA+': 
      prazo_venda = 360
  else:
    prazo_venda = prazo

  melhor_taxa_venda_query = melhores_taxas.query('Indexador == @indexador & Prazo == @prazo_venda')
  melhor_taxa_venda = melhor_taxa_venda_query['Taxa'].values[0]*100
  plan_cliente.loc[i,'melhor_taxa_venda'] = round(melhor_taxa_venda,6)
  #melhor_taxa_venda_produto = plan_cliente.loc[i,'melhor_taxa_venda']

x=0
lista_ativo = ['LCA','LCI']
for i in plan_cliente.fc_indice:
  ativo = plan_cliente.loc[x,'Produto']
  dcs = float(plan_cliente.loc[x,'dias_corridos'])
  melhor_taxa = plan_cliente.loc[x,'melhor_taxa']
  melhor_taxa_venda_produto = plan_cliente.loc[x,'melhor_taxa_venda']
  taxa_cliente = plan_cliente.loc[x,'taxa_emissao']
  if i != 'CDI': 
    ajuste_compra = 8 
    ajuste_venda = 0.50
  else: 
    ajuste_compra = 25 
    ajuste_venda = 1.5
  


  if melhor_taxa <= melhor_taxa_venda_produto:
    
    plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa_venda_produto) + float(ajuste_compra),6)
    plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)

  else:
    plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa) + float(ajuste_compra),6)
    plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)


  if ativo in lista_ativo:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_compra'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_venda'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_emissao'],indexador)*100,6)
  else:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(plan_cliente.loc[x,'taxa_compra'],6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(plan_cliente.loc[x,'taxa_venda'],6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(plan_cliente.loc[x,'taxa_emissao'],6)

  x= x+1

plan_cliente.drop(['melhor_taxa_venda'], axis=1)

temp_pre_cliente = None
temp_cdi_cliente = None
temp_ipca_cliente = None
plan_cliente_cdi = plan_cliente.query('fc_indice == "CDI"').copy().reset_index(drop=True)
if len(plan_cliente_cdi) > 0:
  plan_cliente_cdi = calculo_cdi_venda(pre_interpolado,fer,plan_cliente_cdi)
plan_cliente_pre = plan_cliente.query('fc_indice == "PRE"').copy().reset_index(drop=True)
if len(plan_cliente_pre) > 0:
  plan_cliente_pre = calculo_pre_venda(fer, plan_cliente_pre)
plan_cliente_ipca = plan_cliente.query('fc_indice == "IPCA+"').copy().reset_index(drop=True)
if len(plan_cliente_ipca) > 0:
  plan_cliente_ipca = calculo_ipca_venda(df_ipca_vna,fer,plan_cliente_ipca)


  
  
df_clientes_final = concat(*[plan_cliente_cdi, plan_cliente_pre, plan_cliente_ipca])
df_clientes_final = df_clientes_final.reset_index(drop=True).reset_index(drop=True)
df_clientes_final.rename(columns = {'receita_mesa':'receita_total'}, inplace = True)





df_secundario_input = df_clientes_final.filter(['fc_minemonico','Nome_Emissor','Produto', 'fc_indice','qt_total','Vencimento'
                                                , 'dias_corridos','taxa_venda_sem_grossup','pu_venda','valor_total_venda','receita_assessor' ], axis=1)


df_secundario_input['ROA'] = np.nan
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(str).str.replace(',','.')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(str).str.replace(',','.')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(float)
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(float)
df_secundario_input['ROA'] =  round(df_secundario_input.receita_assessor/ df_secundario_input.valor_total_venda,4)
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(str).str.replace('.',',')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(str).str.replace('.',',')
df_secundario_input['ROA'] = df_secundario_input['ROA'].astype(str).str.replace('.',',')

df_secundario_input = df_secundario_input.drop('receita_assessor',axis=1)

df_secundario_input.columns = ['Instrumento Financeiro','Emissor','produto','indexador','Quantidade Inicial','Vencimento','Dias Corridos - Vencimento','Taxa Negociação','Pu negociação','Valor Financeiro','ROA']






df_assessor_output = df_clientes_final.filter(['Assessor','fc_minemonico','cblc','Produto','fc_indice',
                                               'Nome_Emissor','fd_datemissao','Vencimento','dias_corridos','taxa_emissao', 'qt_total','pu_emissao',
                                               'volume_hoje', 'valor_total_compra','taxa_venda_sem_grossup','valor_total_venda','pu_curva','receita_assessor'
                                               ],axis=1)


taxa = round(14/100,6)

lista_isentos= ['LCI','LCA']
df_assessor_output.pu_curva = df_assessor_output.pu_curva.astype(str).str.replace(',','.')
df_assessor_output.valor_total_venda = df_assessor_output.valor_total_venda.astype(str).str.replace(',','.')
df_assessor_output.valor_total_compra = df_assessor_output.valor_total_compra.astype(str).str.replace(',','.')
#df_assessor_output
#dc_vencimento = (date(data_vencimento.year,data_vencimento.month,data_vencimento.day) -date.today()).days
x=0
for i in range(0,len(df_assessor_output.fd_datemissao)):
  ano = df_assessor_output.fd_datemissao[i].year
  mes = df_assessor_output.fd_datemissao[i].month
  dia = df_assessor_output.fd_datemissao[i].day
  dcs_vencimento = (date.today()-date(ano,mes,dia)).days
  df_assessor_output.at[i,'dcs_vencimento'] = dcs_vencimento

  df_assessor_output.at[i,'volume_hoje'] = round(float(df_assessor_output.pu_curva[i]) * float(df_assessor_output.qt_total[i]),2)
  
  if df_assessor_output.Produto[i] not in lista_isentos:
    df_assessor_output.dropna()
    df_assessor_output.pu_emissao[i] = str(df_assessor_output.pu_emissao[i]).replace(",",".")
    valor_investido = int(float(df_assessor_output.pu_emissao[i])) * df_assessor_output.qt_total[i]
    valor_liquido = float(df_assessor_output.valor_total_compra[i])- ((float(df_assessor_output.valor_total_compra[i]) - valor_investido)*aliquota_ir(dcs_vencimento))
    df_assessor_output.at[i,'Valor Liquido'] = round(valor_liquido,2)
  else:
    df_assessor_output.at[i,'Valor Liquido'] = round(float(df_assessor_output.valor_total_compra[i]),2)

    df_assessor_output.at[i,'breakeven'] = int(breakeven(df_assessor_output.volume_hoje[i],df_assessor_output['Valor Liquido'][i],taxa))

    #df_assessor_output.at[i,'breakeven'] = 0

  #df_assessor_output.at[i,'Valor Liquido'] =  
    x = x+1


df_assessor_output = df_assessor_output.filter(['Assessor','cblc','fc_minemonico','Produto','fc_indice','Nome_Emissor','fd_datemissao',
                                                'Vencimento','dias_corridos','taxa_emissao','qt_total','volume_hoje','valor_total_compra','Valor Liquido',
                                                'breakeven'
                                               ])

#df_assessor_output.columns = ['Assessor','CBLC','Codigo Cetip','Ativo','Indexador','Emissor','Data Emissao','Data Vencimento','DCs Vencimento',
#                              'Taxa Emissao','Quantidade','Financeiro', 'Financeiro Compra - Bruto', 'Financeiro Compra - Liquido','Breakeven(DCs)'

 #        ]









#df_clientes_final.to_excel("df_clientes_final_sec.xlsx")
#files.download('df_clientes_final_sec.xlsx')


#df_secundario_input.to_excel("secundario_input.xlsx")
#files.download('secundario_input.xlsx')


dfs = [df_clientes_final,df_secundario_input,df_assessor_output]
sheets = ['lista_completa', 'layout_heimdall','assessor_tabela']


dfs_tabs(dfs, sheets, f'assessor_trade_idea{datetime.datetime.now()}.xlsx')



In [ ]:
df_assessor_output.pu_emissao.unique()

In [ ]:
x=0
for i in range(0,len(df_assessor_output.fd_datemissao)):
  ano = df_assessor_output.fd_datemissao[i].year
  mes = df_assessor_output.fd_datemissao[i].month
  dia = df_assessor_output.fd_datemissao[i].day
  dcs_vencimento = (date.today()-date(ano,mes,dia)).days
  df_assessor_output.at[i,'dcs_vencimento'] = dcs_vencimento

  df_assessor_output.at[i,'volume_hoje'] = round(float(df_assessor_output.pu_curva[i]) * float(df_assessor_output.qt_total[i]),2)
  
  if df_assessor_output.Produto[i] not in lista_isentos:
    df_assessor_output.pu_emissao[i] = str(df_assessor_output.pu_emissao[i]).replace(",",".")
    valor_investido = int(df_assessor_output.pu_emissao[i]) * df_assessor_output.qt_total[i]
    valor_liquido = float(df_assessor_output.valor_total_compra[i])- ((float(df_assessor_output.valor_total_compra[i]) - valor_investido)*aliquota_ir(dcs_vencimento))
    df_assessor_output.at[i,'Valor Liquido'] = round(valor_liquido,2)
  else:
    df_assessor_output.at[i,'Valor Liquido'] = round(float(df_assessor_output.valor_total_compra[i]),2)

    #df_assessor_output.at[i,'breakeven'] = int(breakeven(df_assessor_output.volume_hoje[i],df_assessor_output['Valor Liquido'][i],taxa))

    df_assessor_output.at[i,'breakeven'] = 0

  #df_assessor_output.at[i,'Valor Liquido'] =  
    x = x+1


df_assessor_output = df_assessor_output.filter(['Assessor','cblc','fc_minemonico','Produto','fc_indice','Nome_Emissor','fd_datemissao',
                                                'Vencimento','dias_corridos','taxa_emissao','qt_total','volume_hoje','valor_total_compra','Valor Liquido',
                                                'breakeven'
                                               ])

df_assessor_output.columns = ['Assessor','CBLC','Codigo Cetip','Ativo','Indexador','Emissor','Data Emissao','Data Vencimento','DCs Vencimento',
                              'Taxa Emissao','Quantidade','Financeiro', 'Financeiro Compra - Bruto', 'Financeiro Compra - Liquido','Breakeven(DCs)'

         ]









#df_clientes_final.to_excel("df_clientes_final_sec.xlsx")
#files.download('df_clientes_final_sec.xlsx')


#df_secundario_input.to_excel("secundario_input.xlsx")
#files.download('secundario_input.xlsx')


dfs = [df_clientes_final,df_secundario_input,df_assessor_output]
sheets = ['lista_completa', 'layout_heimdall','assessor_tabela']


dfs_tabs(dfs, sheets, f'assessor_trade_idea{datetime.datetime.now()}.xlsx')

In [ ]:

df_assessor_output = df_clientes_final.filter(['Assessor','fc_minemonico','cblc','Produto','fc_indice',
                                               'Nome_Emissor','fd_datemissao','Vencimento','dias_corridos','taxa_emissao', 'qt_total','pu_emissao',
                                               'volume_hoje', 'valor_total_compra','taxa_venda_sem_grossup','valor_total_venda','pu_curva','receita_assessor'
                                               ])
taxa = round(14/100,6)

lista_isentos= ['LCI','LCA']
df_assessor_output.pu_curva = df_assessor_output.pu_curva.astype(str).str.replace(',','.')
df_assessor_output.valor_total_venda = df_assessor_output.valor_total_venda.astype(str).str.replace(',','.')
df_assessor_output.valor_total_compra = df_assessor_output.valor_total_compra.astype(str).str.replace(',','.')
#df_assessor_output
#dc_vencimento = (date(data_vencimento.year,data_vencimento.month,data_vencimento.day) -date.today()).days
x=0
for i in range(0,len(df_assessor_output.fd_datemissao)):
  ano = df_assessor_output.fd_datemissao[x].year
  mes = df_assessor_output.fd_datemissao[x].month
  dia = df_assessor_output.fd_datemissao[x].day
  dcs_vencimento = (date.today()-date(ano,mes,dia)).days
  df_assessor_output.at[i,'dcs_vencimento'] = dcs_vencimento

  df_assessor_output.at[i,'volume_hoje'] = round(float(df_assessor_output.pu_curva[i]) * float(df_assessor_output.qt_total[i]),2)
  
  if df_assessor_output.Produto[i] not in lista_isentos:
    valor_investido = int(df_assessor_output.pu_emissao[i]) * df_assessor_output.qt_total[i]
    valor_liquido = float(df_assessor_output.valor_total_compra[i])- ((float(df_assessor_output.valor_total_compra[i]) - valor_investido)*aliquota_ir(dcs_vencimento))
    df_assessor_output.at[i,'Valor Liquido'] = round(valor_liquido,2)
  else:
    df_assessor_output.at[i,'Valor Liquido'] = round(float(df_assessor_output.valor_total_compra[i]),2)

  df_assessor_output.at[i,'breakeven'] = int(breakeven(df_assessor_output.volume_hoje[i],df_assessor_output['Valor Liquido'][i],taxa))



  #df_assessor_output.at[i,'Valor Liquido'] =  
  x = x+1


df_assessor_output = df_assessor_output.filter(['Assessor','cblc','fc_minemonico','Produto','fc_indice','Nome_Emissor','fd_datemissao',
                                                'Vencimento','dias_corridos','taxa_emissao','qt_total','volume_hoje','valor_total_compra','Valor Liquido',
                                                'breakeven'
                                               ])

df_assessor_output.columns = ['Assessor','CBLC','Codigo Cetip','Ativo','Indexador','Emissor','Data Emissao','Data Vencimento','DCs Vencimento',
                              'Taxa Emissao','Quantidade','Financeiro', 'Financeiro Compra - Bruto', 'Financeiro Compra - Liquido','Breakeven(DCs)'

         ]



# Ararinha

In [ ]:
#plan_cliente = files.upload()
#plan_cliente = pd.read_excel('assessoria_santander2610.xlsx')
#plan_cliente = pd.read_excel('011122_Ararinha - pricing.xlsx')
#plan_cliente = plan_cliente.query('fc_indice != "CDI"')
#plan_cliente.reset_index(drop=True,inplace=True)
#melhores_taxas = taxa_venda_()
#plan_cliente = plan_cliente.query('volume_hoje >= 10000')
plan_ararinha = 'Ajuste - Posição Ararinha'
#sh_ararinha = gc.open(plan_ararinha)
plan = gc.open(plan_ararinha).worksheet('Ararinha_Posi')
plan_cliente = pd.DataFrame(plan.get_all_records())
plan_cliente.Vencimento = plan_cliente.Vencimento.apply(lambda _: datetime.datetime.strptime(_,"%d/%m/%Y").date())
plan_cliente.fd_datemissao = plan_cliente.fd_datemissao.apply(lambda _: datetime.datetime.strptime(_,"%d/%m/%Y").date())
plan_cliente['taxa_emissao'] = plan_cliente['taxa_emissao'].astype(str).str.replace(',','.')
plan_cliente['pu_emissao'] = plan_cliente['pu_emissao'].astype(str).str.replace(',','.')
plan_cliente['taxa_emissao'] = pd.to_numeric(plan_cliente['taxa_emissao']) 
plan_cliente['pu_emissao'] = pd.to_numeric(plan_cliente['pu_emissao'])

plan_cliente.reset_index(drop=True ,inplace=True)
print(len(plan_cliente))
#plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('CDI','DI')
plan_cliente.fc_indice = plan_cliente.fc_indice.astype(str).str.replace('IPCA','IPCA+')

for i in range(0,len(plan_cliente.dias_corridos)):
  print(i)
  ano = plan_cliente.Vencimento[i].year
  mes = plan_cliente.Vencimento[i].month
  dia = plan_cliente.Vencimento[i].day

  plan_cliente.at[i,'dias_corridos'] = (date(ano,mes,dia) -date.today()).days



plan_cliente = plan_cliente.query('dias_corridos > 30').reset_index(drop=True)
plan_cliente.melhor_taxa = np.nan
plan_cliente.melhor_taxa_venda = np.nan
plan_cliente.taxa_venda = np.nan
plan_cliente.melhor_taxa_normalizada = np.nan
plan_cliente.taxa_compra = np.nan
plan_cliente.taxa_emissao_normalizada = np.nan

for i in range(0,len(plan_cliente.fc_indice)):
  indexador = plan_cliente.loc[i,'fc_indice']
  prazo = plan_cliente.loc[i,'dias_corridos']
  ativo = plan_cliente.loc[i,'Produto']
  dcs = plan_cliente.loc[i,'dias_corridos']
  prazo = 360 * round(prazo /360)
  if prazo < 30 : prazo = 30 
  #elif prazo < 360 and indexador == 'IPCA+': prazo = 360
  melhor_taxa = melhores_taxas_normalizada.query('Indexador == @indexador & Prazo == @prazo')

  valor = melhor_taxa['Taxa'].values[0]
  plan_cliente.loc[i,'melhor_taxa'] = round(valor*100,6)
  #melhor_taxa = plan_cliente.loc[i,'melhor_taxa']


  if prazo < 360 and indexador !=	'IPCA+': 
      prazo_venda = 180 
  elif prazo < 360 and indexador == 'IPCA+': 
      prazo_venda = 360
  else:
    prazo_venda = prazo

  melhor_taxa_venda_query = melhores_taxas.query('Indexador == @indexador & Prazo == @prazo_venda')
  melhor_taxa_venda = melhor_taxa_venda_query['Taxa'].values[0]*100
  plan_cliente.loc[i,'melhor_taxa_venda'] = round(melhor_taxa_venda,6)
  #melhor_taxa_venda_produto = plan_cliente.loc[i,'melhor_taxa_venda']

x=0
lista_ativo = ['LCA','LCI']
for i in plan_cliente.fc_indice:
  ativo = plan_cliente.loc[x,'Produto']
  dcs = float(plan_cliente.loc[x,'dias_corridos'])
  melhor_taxa = plan_cliente.loc[x,'melhor_taxa']
  melhor_taxa_venda_produto = plan_cliente.loc[x,'melhor_taxa_venda']
  taxa_cliente = plan_cliente.loc[x,'taxa_emissao']
  if i != 'CDI': 
    ajuste_compra = 5 
    ajuste_venda = 0.5
  else: 
    ajuste_compra = 9 
    ajuste_venda = 1
  


  if melhor_taxa <= melhor_taxa_venda_produto:
    
    plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa_venda_produto) + float(ajuste_compra),6)
    plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)

  else:
    plan_cliente.loc[x,'taxa_compra'] = round(float(melhor_taxa) + float(ajuste_compra),6)
    plan_cliente.loc[x,'taxa_venda'] = round(float(melhor_taxa_venda_produto) + float(ajuste_venda),6)


  if ativo in lista_ativo:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_compra'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_venda'],indexador)*100,6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(grossup_nao_isento(pre_interpolado, implicita_interpolada,fer, ativo,dcs,plan_cliente.loc[x,'taxa_emissao'],indexador)*100,6)
  else:
    plan_cliente.loc[x,'taxa_compra_sem_grossup'] = round(plan_cliente.loc[x,'taxa_compra'],6)
    plan_cliente.loc[x,'taxa_venda_sem_grossup'] = round(plan_cliente.loc[x,'taxa_venda'],6)
    plan_cliente.loc[x,'taxa_emissao_sem_grossup'] = round(plan_cliente.loc[x,'taxa_emissao'],6)

  x= x+1

plan_cliente.drop(['melhor_taxa_venda'], axis=1)

temp_pre_cliente = None
temp_cdi_cliente = None
temp_ipca_cliente = None
plan_cliente_cdi = plan_cliente.query('fc_indice == "CDI"').copy().reset_index(drop=True)
if len(plan_cliente_cdi) > 0:
  plan_cliente_cdi = calculo_cdi_venda(pre_interpolado,fer,plan_cliente_cdi)
plan_cliente_pre = plan_cliente.query('fc_indice == "PRE"').copy().reset_index(drop=True)
if len(plan_cliente_pre) > 0:
  plan_cliente_pre = calculo_pre_venda(fer, plan_cliente_pre)
plan_cliente_ipca = plan_cliente.query('fc_indice == "IPCA+"').copy().reset_index(drop=True)
if len(plan_cliente_ipca) > 0:
  plan_cliente_ipca = calculo_ipca_venda(df_ipca_vna,fer,plan_cliente_ipca)


  
  
df_clientes_final = concat(*[plan_cliente_cdi, plan_cliente_pre, plan_cliente_ipca])
df_clientes_final = df_clientes_final.reset_index(drop=True).reset_index(drop=True)
df_clientes_final.rename(columns = {'receita_mesa':'receita_total'}, inplace = True)





df_secundario_input = df_clientes_final.filter(['fc_minemonico','Nome_Emissor','Produto', 'fc_indice','qt_total','Vencimento'
                                                , 'dias_corridos','taxa_venda_sem_grossup','pu_venda','valor_total_venda','receita_assessor' ], axis=1)


df_secundario_input['ROA'] = np.nan
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(str).str.replace(',','.')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(str).str.replace(',','.')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(float)
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(float)
df_secundario_input['ROA'] =  round(df_secundario_input.receita_assessor/ df_secundario_input.valor_total_venda,4)
df_secundario_input.valor_total_venda = df_secundario_input.valor_total_venda.astype(str).str.replace('.',',')
df_secundario_input.receita_assessor = df_secundario_input.receita_assessor.astype(str).str.replace('.',',')
df_secundario_input['ROA'] = df_secundario_input['ROA'].astype(str).str.replace('.',',')

df_secundario_input = df_secundario_input.drop('receita_assessor',axis=1)

df_secundario_input.columns = ['Instrumento Financeiro','Emissor','produto','indexador','Quantidade Inicial','Vencimento','Dias Corridos - Vencimento','Taxa Negociação','Pu negociação','Valor Financeiro','ROA']






df_assessor_output = df_clientes_final.filter(['Assessor','fc_minemonico','cblc','Produto','fc_indice',
                                               'Nome_Emissor','fd_datemissao','Vencimento','dias_corridos','taxa_emissao', 'qt_total','pu_emissao',
                                               'volume_hoje', 'valor_total_compra','taxa_venda_sem_grossup','valor_total_venda','pu_curva','receita_assessor'
                                               ],axis=1)


taxa = round(14/100,6)

lista_isentos= ['LCI','LCA']
df_assessor_output.pu_curva = df_assessor_output.pu_curva.astype(str).str.replace(',','.')
df_assessor_output.valor_total_venda = df_assessor_output.valor_total_venda.astype(str).str.replace(',','.')
df_assessor_output.valor_total_compra = df_assessor_output.valor_total_compra.astype(str).str.replace(',','.')
#df_assessor_output
#dc_vencimento = (date(data_vencimento.year,data_vencimento.month,data_vencimento.day) -date.today()).days
x=0
for i in range(0,len(df_assessor_output.fd_datemissao)):
  ano = df_assessor_output.fd_datemissao[i].year
  mes = df_assessor_output.fd_datemissao[i].month
  dia = df_assessor_output.fd_datemissao[i].day
  dcs_vencimento = (date.today()-date(ano,mes,dia)).days
  df_assessor_output.at[i,'dcs_vencimento'] = dcs_vencimento

  df_assessor_output.at[i,'volume_hoje'] = round(float(df_assessor_output.pu_curva[i]) * float(df_assessor_output.qt_total[i]),2)
  
  if df_assessor_output.Produto[i] not in lista_isentos:
    df_assessor_output.dropna()
    df_assessor_output.pu_emissao[i] = str(df_assessor_output.pu_emissao[i]).replace(",",".")
    valor_investido = int(float(df_assessor_output.pu_emissao[i])) * df_assessor_output.qt_total[i]
    valor_liquido = float(df_assessor_output.valor_total_compra[i])- ((float(df_assessor_output.valor_total_compra[i]) - valor_investido)*aliquota_ir(dcs_vencimento))
    df_assessor_output.at[i,'Valor Liquido'] = round(valor_liquido,2)
  else:
    df_assessor_output.at[i,'Valor Liquido'] = round(float(df_assessor_output.valor_total_compra[i]),2)

    df_assessor_output.at[i,'breakeven'] = int(breakeven(df_assessor_output.volume_hoje[i],df_assessor_output['Valor Liquido'][i],taxa))

    #df_assessor_output.at[i,'breakeven'] = 0

  #df_assessor_output.at[i,'Valor Liquido'] =  
    x = x+1


df_assessor_output = df_assessor_output.filter(['Assessor','cblc','fc_minemonico','Produto','fc_indice','Nome_Emissor','fd_datemissao',
                                                'Vencimento','dias_corridos','taxa_emissao','qt_total','volume_hoje','valor_total_compra','Valor Liquido',
                                                'breakeven'
                                               ])

#df_assessor_output.columns = ['Assessor','CBLC','Codigo Cetip','Ativo','Indexador','Emissor','Data Emissao','Data Vencimento','DCs Vencimento',
 #                             'Taxa Emissao','Quantidade','Financeiro', 'Financeiro Compra - Bruto', 'Financeiro Compra - Liquido','Breakeven(DCs)'

  #       ]









#dfs = [df_clientes_final,df_secundario_input,df_assessor_output]
#sheets = ['lista_completa', 'layout_heimdall','assessor_tabela']


#dfs_tabs(dfs, sheets, f'Ararinha - {datetime.datetime.now()}.xlsx')

df_secundario_input.dropna(inplace=True)
pu_compra = gc.open(plan_ararinha).worksheet('Run de Ativos')
plan_pu_compra = pd.DataFrame(pu_compra.get_all_records())
ajuste_pu_compra = plan_pu_compra[['CodCustodia','PUCompra']]
ajuste_pu_compra['PUCompra'] = ajuste_pu_compra['PUCompra'].astype(str).str.replace(',','.')
ajuste_pu_compra['PUCompra'] = pd.to_numeric(ajuste_pu_compra['PUCompra']) 
ajuste_pu_compra.rename(columns = {'CodCustodia':'Instrumento Financeiro'}, inplace = True)
teste = df_secundario_input.merge(ajuste_pu_compra, on='Instrumento Financeiro', how='left')
teste['Pu negociação'] = teste['Pu negociação'].astype(str).str.replace(',','.',)
teste['Pu negociação'] = pd.to_numeric(teste['Pu negociação'])
teste['check_pu_compra'] = teste['Pu negociação'] < teste['PUCompra']
plan_final = teste.query('check_pu_compra == False')

plan_final.dropna(inplace=True)
#plan_final.to_excel(f'Ararinha - {datetime.datetime.now()}.xlsx', encoding ='utf-8')
plan_subir_sheets = plan_final[['Instrumento Financeiro', 'produto','Emissor','indexador','Vencimento','Quantidade Inicial','Taxa Negociação','Pu negociação','Valor Financeiro','ROA']]

dfs = [plan_final,plan_subir_sheets]
sheets = ['Planilha Completa', 'layout_Ararinha']


dfs_tabs(dfs, sheets, f'Ararinha - {datetime.datetime.now()}.xlsx')
